# *입력되는 엑셀의 기본 조건

## 1. 제목 행은 총 4행 > 템플릿 파일들 기준으로 맞추자!
- 1행 : 2023년_ [4-마. 재적 학생 현황_] 같은 설명
- 2~3행 : 빈행 (체크 필요)
- 4행 ~ 7행 : 제목행 (셀들은 병합되어있으며 변경됨)

## 2. 삭제되는 공통열은 university 테이블 참조
- '설립구분'
- '학교종류'
- '지역'
- '상태'

## 3. 엑셀의 컬럼명과 일치하는 컬럼명으로 자동 매칭되도록...

In [13]:
import os, sys
# 판다스
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd


#################################################
# 디버그 모드 설정
#################################################
def _dbg_(a='', b='', c='', d='', e=''):
    print('[dbg]', sys._getframe(1).f_code.co_name, ' : ', a, b, c, d, e)

In [14]:
#################################################
# DB 접속 설정
#################################################

#%pip install pymysql
import pymysql

def db_setting(ip, id, pwd, dbname):
    conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
    curs = conn.cursor(pymysql.cursors.DictCursor)

    if conn == None:
        print('DB 연결 오류')
    elif curs == None:
        print('curs 연결 오류')
        
    return conn, curs



In [23]:
#################################################
# 물리적 파일명 검증, 각 디렉토리에서 일치하는 파일명 기준으로 일괄 작업 수행
#################################################

#디렉토리의 전체 파일명 읽기
def read_filename(dir): 
    file_list = os.listdir(dir)
    return file_list

#전체 경로에서 파이명만 추출
def extract_filename(full_name): 
    file_name = os.path.basename(full_name)
    return file_name


#################################################
# 템플릿데이터와 입력된 데이터 검증
# return doc_title : 해당 문서명을 기준으로 테이블 생성여부 확인
#################################################
def compare_schema(template, excel, sheet_name): 
    _dbg_(template, excel, sheet_name)
    # excel file load
    #%pip install openpyxl
    df_template = pd.read_excel(template, sheet_name = sheet_name) #템플릿과 일치여부를 확인하자
    df_excel = pd.read_excel(excel, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df_template = df_template.fillna('None')
    df_excel = df_excel.fillna('None')
    
    # 1행 제목행 비교
    df_template_header = list(df_template.columns) #템플릿 헤더
    df_excel_header = list(df_excel.columns) #df 헤더
    
    #_dbg_(df_excel_header[0]) #['2023년_ [4-다. 신입생 충원 현황_]        doc_title :  7 23 2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] +1 -1
    doc_title = df_excel_header[0]
    pos_1 = doc_title.find('. ')
    pos_2 = doc_title.find('_]')
    
    if pos_1 <= 0 or pos_2 <= 0:
        print('[종료] 엑셀 문서 제목 확인 필요 ', df_excel_header[0])
        sys.exit(0)
        
    doc_title = doc_title[pos_1+2:pos_2]
    _dbg_('doc_title : ', pos_1, pos_2, df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    _dbg_('작업 문서명 ', df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    
    df_template_header[0].find('[', 0)
    df_excel_header[0].find('[', 0)

    df_template_header[0] = '' #첫행 첫열은 0000년_ [2-가-1. 전공과목 성적 분포_]
    df_excel_header[0] = ''
    
    #_dbg_('df_template_header', df_template_header)
    #_dbg_('df_excel_header', df_excel_header)
    if df_template_header != df_excel_header:
        print('[종료] : 첫줄 데이터 구조가 다름(1행)')
        sys.exit(0)
        #return 0, 0
        
    #_dbg_('df_template', type(df_template), df_template)
    #_dbg_('df_template.shape[0]', type(df_template.shape[0]), df_template.shape[0])


    # 2행 이하의 제목행 비교
    df_template_shape0 = df_template.shape[0]
    for n in range(df_template_shape0): #엑셀의 첫 4행정도만 확인
        template_rows_list = list(df_template.loc[n]) #컬럼 시리즈를 리스트로 변경
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        #print('df_templ 리스트 : ', type(template_rows_list), len(template_rows_list), template_rows_list)
        #print('df_excel 리스트 : ', type(excel_rows_list), len(excel_rows_list), excel_rows_list)
        
        #_dbg_(n, 'template_rows_list', template_rows_list)
        #_dbg_(n, 'excel_rows_list', excel_rows_list)        

        if template_rows_list != excel_rows_list:
            print('[종료] : {}번째 줄 데이터 구조가 다름(1행)'.format(n))
            sys.exit(0)
            #return 0, 0, 0
   
    return df_excel, doc_title, df_template_shape0



    
    

#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row1(df_excel, df_template_shape0=0):
    _dbg_('df_template_shape0 : ', df_template_shape0)

    #_dbg_('df_excel.info() : ', type(df_excel.info()), df_excel.info())
    
    header_info = df_excel.columns.tolist()
    _dbg_('header_info : ', type(header_info), header_info)
    
    
    new_header = []
    new_header_cnt = 0
    
    ## 학교 > 학교명 등 특정잘못된 제목부터 변경
    for n in range(df_template_shape0): #템플릿 엑셀기준 행정보 확인
        #_dbg_(df_template_shape0, n)
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        _dbg_(n, excel_rows_list)
        for m in range(len(excel_rows_list)): #각 열 순회
            cell = excel_rows_list[m]
            if cell == '기준연도':
                new_header.append(cell)
                new_header_cnt += 1
            elif cell == '학교종류':
                new_header.append(cell)
                new_header_cnt += 1
            elif cell == '설립구분':
                new_header.append(cell)
                new_header_cnt += 1
            elif cell == '지역':
                new_header.append(cell)
                new_header_cnt += 1
            elif cell == '상태':
                new_header.append(cell)
                new_header_cnt += 1
            elif cell == '학교명' or cell == '학교':
                new_header.append('학교명')
                new_header_cnt += 1
                
    #나머지 new_header 정의
    for m in range(len(excel_rows_list)-new_header_cnt): #각 열 순회
        new_header.append('new-header')

    #새로운 헤더 지정
    df_excel.columns = new_header

    #나머지 제목행들 삭제
    for n in range(df_template_shape0):
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        _dbg_('{}행 - 삭제 : {}'.format(n, excel_rows_list))
        df_excel = df_excel.drop(n) #빈행 삭제 
        #remove_cnt = remove_cnt + 1

    # # 제목행 후 빈행 제거
    # remove_cnt = 0
    # for n in range(df_template_shape0): #템플릿 엑셀기준 행정보 확인
    #     excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
    #     #_dbg_(n, excel_rows_list)
    #     #del 0['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', ...]  
    #     #del 1['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', ...]  
    #     #2['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '총입학자수', '출신학교 유형별 입학자수 및 비율', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '출신학교 지역정보별 입학자수 및 비율', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']  
    #     #3['None', 'None', 'None', 'None', 'None', 'None', 'None', '일반고', 'None', '특수목적고', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '특성화고', 'None', '자율고', 'None', 'None', 'None', '기타', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '특별시', 'None', '광역시·특별자치시', 'None', '중소도시', 'None', '읍면', 'None', '특수', 'None', '기타', 'None', '소계', 'None']  
    #     #4['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '과학고', 'None', '외고·\n국제고', 'None', '예술·\n체육고', 'None', '산업수요맞춤형\n고등학교', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '영재학교', 'None', '외국인학교', 'None', '외국고등학교', 'None', '검정고시', 'None', '그 외 기타', 'None', '소계', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']  
    #     #del 5['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', ...]  
    #     #6['None', 'None', 'None', 'None', 'None', 'None', 'None', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '사립', 'None', '공립', 'None', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율', '학생수', '비율']  
    #     #7['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '학생수', '비율', '학생수', '비율', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']  
    #     #_dbg_(type(excel_rows_list), len(excel_rows_list))

    #     cnt_none = 0 #None으로 할당된 셀 카운트
    #     for excel_cell in excel_rows_list:
    #         if excel_cell == 'None':
    #             cnt_none += 1
                
    #     if cnt_none >= len(excel_rows_list):
    #         _dbg_('{}행 - None이므로 삭제'.format(n))
    #         df_excel = df_excel.drop(n) #빈행 삭제 
    #         remove_cnt = remove_cnt + 1


    # #타이틀 컬럼명 재지정
    # if remove_cnt >= 1:
    #     df_excel.columns = list(df_excel.loc[remove_cnt-1])
    #     _dbg_('df_excel.columns : ', remove_cnt, df_excel.columns)
        
    df_excel = reset_index(df_excel)



    ## 삭제 후 정리된 행정보 출력
    for n in range(df_template_shape0): #템플릿 엑셀기준 행정보 확인
        #_dbg_(df_template_shape0, n)
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        _dbg_(n, excel_rows_list)





# header_mapping = {
#     'old_header1': 'new_header1',
#     'old_header2': 'new_header2',
#     # Add more mappings as needed
# }

# # Rename the headers using the dictionary
# df_excel.rename(columns=header_mapping, inplace=True)





    return df_excel

#################################################
# df 헤더명이 다른 경우가 있음 학교명 vs 학교...
#################################################
def refine_header(df_excel):
    # _dbg_()

    # df_excel_header = list(df_excel.columns) #df 헤더

    # #엑셀의 [1,1]은 2023년_ [4-자. 신입생의 출신 고등학교 유형별 현황_]
    # df_excel_header[0].find('[', 0)
    # df_excel_header[0] = '기준연도'
    
    # for n in range(len(df_excel_header)):
    #     if df_excel_header[n] == '학교':
    #         df_excel_header[n] = '학교명'
            
    # _dbg_('df_excel_header', df_excel_header)
    
    # #타이틀 컬럼명 재지정
    # df_excel.columns = list(df_excel_header)
    
    return df_excel


#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row2(df_excel):
    _dbg_()

    find_year = ''
    index_values = list(df_excel.index.values)
    _dbg_('index_values : ', index_values)
    for idx in index_values:
        cell_year = df_excel.loc[idx, '기준연도']
        cell_univ = df_excel.loc[idx, '학교명']
        #print('remove_empty_row2 (cell_univ) : ', cell_univ)
        cell_type = df_excel.loc[idx, '학교종류']
        
        if cell_year == '기준연도' and cell_univ == '학교명' and cell_type == '학교종류':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거
        elif cell_year == '기준연도' and cell_univ == '학교명' or cell_univ == '학교':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거   
        
                     

        if cell_year == 'None' and cell_univ == 'None' and cell_type == 'None':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목 3행중 2행 삭제
        elif cell_year != 'None':
            find_year = cell_year
    
    df_excel = reset_index(df_excel)
    #print('찾은 년도 : {}'.format(find_year))
    
    
    header_info = df_excel.columns.tolist()
    _dbg_('header_info : ', type(header_info), header_info)

    
    return df_excel, find_year
        

#################################################
# df_excel에서 [학교종류 | 설립구분 | 지역 | 상태] 열 삭제
#################################################
def remove_univ_col(df_excel):
    _dbg_()
    
    df_excel = df_excel.drop(['설립구분'], axis = 1)
    df_excel = df_excel.drop(['학교종류'], axis = 1)
    df_excel = df_excel.drop(['지역'], axis = 1)
    df_excel = df_excel.drop(['상태'], axis = 1)

    return df_excel
            
#################################################
# 연도 찾아서 채우기
#################################################
def fill_year_col(df_excel, find_year, col_name = '기준연도',):
    _dbg_()
    
    #column_values = list(df_excel[col_name])
    #print(column_values)
        
    #df_excel = df_excel[col_name].fillna(cell, inplace=True)
    df_excel[col_name].replace('None', find_year, inplace=True)
    
    print(df_excel.head(5))
    return df_excel


#################################################
# 삭제된 df index 재설정
#################################################
def reset_index(df):
    #삭제된 줄만큼 루프를 더 돌던지.... 인덱스 값을 재조정하던지
    df = df.reset_index(drop=True) #(inplace=True)
    return df
       

#################################################
# 가천대학교 None 식으로 되어있는 학교이름을 위쪽 이름으로 채우기
# 0000년 _대학_2-가-1. 전공과목 성적 분포_학교별자료.xlsx 에만 적용됨
#################################################
def fill_univ_name(df_excel, col_name = '학교명'):
    _dbg_()
    column_values = list(df_excel[col_name])
    #print(column_values)
    
    cur_univ = ''
    prev_univ = ''
    index_values = list(df_excel.index.values)
    
    idx_max = len(index_values)-1

    for idx in index_values:
        
        #print(idx, idx_max)
        if idx >= idx_max:
            break
        
        #cell_year = df_excel.loc[idx, '기준연도']
        #prev_univ = df_excel.loc[idx-1, col_name]
        cur_univ = df_excel.loc[idx, col_name]
        next_univ = df_excel.loc[idx+1, col_name]
        #cell_type = df_excel.loc[idx, '학교종류']
        
        #print(idx, cur_univ, next_univ)
        
        if cur_univ != '' and next_univ == 'None':
            df_excel.loc[idx+1, col_name] = cur_univ
            #print(cur_univ, next_univ)
            
    return df_excel

#################################################
# 대학명에서 좌우공백 제거... 전체 공백을 지우는게 유리할수도...
#################################################
def strip_univ_name(df_excel, col_name='학교명'):
    _dbg_()
    
    for n in range(len(df_excel)): #0~491 데이터프레임에서 학교명을 코드명으로 변경
        df_excel_univname = str(df_excel.loc[n, col_name])
        
        #df_excel_univname = df_excel_univname.strip() 전체
        df_excel_univname = df_excel_univname.lstrip()
        df_excel_univname = df_excel_univname.rstrip()
        
    return df_excel



#################################################
# university테이블에서 특정 또는 가장 큰 코드값 읽기
#################################################
def get_univ_maxcode(curs):
    _dbg_()
    
    sql = 'SELECT {} FROM {}'.format('univ_cd', 'university') #university 테이블에서 가장큰 univ_code값을 가져오기 위함
    curs.execute(sql)
    code = curs.fetchall() #university에 등록된 대학교 목록
    
    max_code_val = 0
    
    for cd in code:
        #print('학교코드', type(cd), cd, cd['univ_cd'])
        
        code_str = cd['univ_cd']
        if code_str.find('-') == -1: #특수 기호 없다면
            max_code_val = max(max_code_val, int(code_str))
            
    #기존 테이블에 최대값은 10000x으로 되기 때문에 문자열로해도 100000x
    max_code_val += 1
    return str(max_code_val)
    
#################################################
# 유틸 : 문자열내 공백 제거, 문자열 교체
#################################################
def refine_univ_name(str):
    # A단계에서 처리가 안되는경우 2차 예외처리
    # 강릉원주대학교 vs 국립강릉원주대학교 와 같은 식으로 프리픽스와 포스트가 다른 경우가 있다.
    # 하나의 코드명으로 처리해야하기 때문에 유사한 대학명을 동일하게 매핑해야함            
    # 1. 국립 : 국립강릉원주대학교, 경상국립대학교와 같은 패턴, # 2. 숫자로 된 캠퍼스는 숫자만 일치해도 동일하게 처리

    str = str.lstrip()
    str = str.rstrip()
    str = str.replace('국립', '') #국립제거
    str = str.replace(' ', '') #띄어쓰기제거
    return str


#################################################
# 데이터 프레임의 학교명을 코드명으로 변경
# - 학교명과 일치하는 univ_cd를 찾아서 데이터프레임 정리
# conn, dbname, excel_file, sheet_name : university 테이블에 등록되지 않은 학교를 발견한 경우 다시 정보를 읽어오기 위함
# curs와 df_excel 만 실제로 필요함
#################################################
def set_univ_code(curs, conn, df_excel, dbname, excel_file, sheet_name):
    _dbg_(excel_file, sheet_name)
    
    print('[IN] set_univ_code(curs, conn, df_excel, dbname, excel_file, sheet_name)')

    add_new_univ_code = False #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함

    sql_select = 'SELECT {}, {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'univ_name', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql_select)

    univ_data = curs.fetchall() #university에 등록된 대학교 목록

    # print(type(univ_data), univ_data)
    # print(type(univ_data[0]), univ_data[0])
    # print(type(univ_data[0]['univ_name_arlimi']), univ_data[0]['univ_name_arlimi'], univ_data[0]['univ_cd'])
    #conn.commit()

    #print('len(df_excel) : ' , len(df_excel))
    for n in range(len(df_excel)): #엑셀 데이터프레임에서 학교명을 추출

        df_excel_univname = str(df_excel.loc[n, '학교명'])
        #_dbg_('refine 전 : ', df_excel_univname)
        df_excel_univname = refine_univ_name(df_excel_univname)
        #_dbg_('refine 후 : ', df_excel_univname)
                    
        find_univ_code = False #university 테이블에 대학코드가 등록되지 않는 경우도 있음
        
        # A.엑셀대학명과 university 테이블의 코드명 매핑 - 대부분은 여기서 완료됨
        for i in range(len(univ_data)): #466 university 테이블에서 검색
            #if df_excel_univname == '화성의과학대학교':
            #print(n, df_excel_univname)

            univ_name_arlimi = univ_data[i]['univ_name_arlimi']
            univ_name_arlimi = refine_univ_name(univ_name_arlimi)
            
            univ_name_general = univ_data[i]['univ_name']
            univ_name_general = refine_univ_name(univ_name_general)

            #(알리미 또는 univ_name명칭) 엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
            if df_excel_univname == univ_name_arlimi:
                #dat = '알리미 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
                #_dbg_(dat)
                _dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
                df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']
                find_univ_code = True
                break
            # elif df_excel_univname == univ_name_general:
            #     dat = '일반 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
            #     _dbg_(dat)
            #     _dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
            #     df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']                
            #     find_univ_code = True
            #     break

        
        #어쩌다 한번씩 벌어질 일이기 때문에 그냥 재작업하자
        if find_univ_code == False: #university 테이블에 등록되지 않은 학교를 발견, DB에 등록, ex)신경주대학교
            _dbg_('if find_univ_code == False', n, df_excel_univname)
            df = pd.read_excel(excel_file, sheet_name = sheet_name)
            df = df.fillna('None')
            df = remove_empty_row1(df) #
            df = refine_header(df)
            df, find_year = remove_empty_row2(df)
            
            
            index_values = list(df_excel.index.values)
            for idx in index_values:
                #cell_year = df_excel.loc[idx, '기준연도']
                cell_univ_name = df.loc[idx, '학교명']
                cell_univ_name = cell_univ_name.lstrip()
                cell_univ_name = cell_univ_name.rstrip()
                
                _dbg_(idx, cell_univ_name)
                
                
                if cell_univ_name == df_excel_univname: #신규 대학 발견, 관련 정보를 university 테이블에 등록
                    _dbg_('신규 대학 발견, 관련 정보를 university 테이블에 등록', idx, cell_univ_name, df_excel_univname)
                    cell_type = df.loc[idx, '학교종류']
                    cell_estblish = df.loc[idx, '설립구분']
                    cell_status = df.loc[idx, '상태']
                    cell_locale = df.loc[idx, '지역']
                    
                    max_code = get_univ_maxcode(curs)
                    
                    #'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                    
                    col_name_str = '`univ_cd`, `univ_name_arlimi`, `univ_name`, `univ_name_en`, `univ_type`, `establish`, `status`, `campus`, `campus_name`, `locale`, `homepage`, `tel`, `fax`'
                    field_data_str = "'{}', '{}', '{}', 'None', '{}', '{}', '{}', 'None', 'None', '{}', 'None', 'None', 'None'".format(max_code, cell_univ_name, cell_univ_name, cell_type, cell_estblish, cell_status, cell_locale)
                    sql_new_univ = 'insert into {}.{}({}) values({})'.format(dbname, 'university', col_name_str, field_data_str)
                    
                    curs.execute(sql_new_univ)
                    conn.commit()
                    
                    #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함
                    sql_select = 'SELECT {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
                    _dbg_(sql_select)
                    curs.execute(sql_select)
                    univ_data = curs.fetchall() #university에 등록된 대학교 목록
                    
                    
                    #엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
                    #if df_excel_univname == univ_data[i]['univ_name_arlimi']:
                        #print(univ_data[i]['univ_name_arlimi'], univ_data[i]['univ_cd'])
                        
                    df_excel.loc[n, '학교명'] = max_code
     
    return df_excel



#################################################
# 기준이 되는 학교정보 테이블 입력, 외부데이터와는 달리 자체 생성 데이터
#################################################
def create_university(conn, curs, dbname, table, excel_path, sheet_name, table_reset = False, mode_sql = 'I/U/D'):
    _dbg_(table)
    df = pd.read_excel(excel_path, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df = df.fillna('None')

    #df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
    #df.columns('a', 'b') 

    cols_ko = list(df.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df.drop(0) #특정행 삭제
    
    # db truncate tables, 기존 데이터를 삭제 후 인서트
    if table_reset == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table)
        curs.execute(sql_truncate)
        conn.commit()
        print('{}.{} 테이블 삭제 완료'.format(dbname, table), sql_truncate)
    
    # data insert to db
    field_names = ''
    field_data = ''


    for n in range(len(cols_en)):
        if n == 0 :
            field_names = cols_en[n]
            field_data = '%s'
        else:
            field_names = field_names + ', ' + cols_en[n]
            field_data = field_data + ', ' + '%s'
            
        #입력데이터 중 좌우측 공백제거
        field_data = field_data.lstrip()
        field_data = field_data.rstrip()
        
        
    # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    # print(sql_insert)

    univ_name_arlimi_date = ''
    univ_cd=''
    if mode_sql == 'I': #insert mode
        sql = 'insert into {}.{}({}) values({})'.format(dbname, table, field_names, field_data)
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드 >>>> 변경된 데이터만 찾아서 향후 별도 처리
        sql = 'update {}.{} set {}={} where univ_cd={}'.format(dbname, table, 'univ_name_arlimi', univ_name_arlimi_date, univ_cd)
    
    print('1번 : ', sql)
    
    for idx in range(len(df)):
        #print(idx, sql)
        curs.execute(sql, tuple(df.values[idx]))
    conn.commit()
    
    if mode_sql == 'I': #insert mode
        print(idx, '개의 데이터 입력 성공')
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드
        print(idx, '개의 데이터 업데이트 성공')
    
    # UPDATE cosshubtest.university
	# SET univ_name_arlimi='None-'
	# WHERE univ_cd='0002653'
 
 
 


In [16]:
#!/usr/bin/env python
# # coding: utf-8

#################################################
# 테이블 스키마 정의
#################################################
table_dic = {}

# 학교 코드 정보 테이블
table_dic['university'] = \
    "CREATE TABLE `university` (\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `univ_name_arlimi` varchar(100) DEFAULT NULL COMMENT '대학명(알리미기준)',\
    `univ_name` varchar(100) NOT NULL COMMENT '대학명(일반명칭)',\
    `univ_name_en` varchar(100) NOT NULL COMMENT '대학명(영문)',\
    `univ_type` varchar(50) NOT NULL COMMENT '학교종류',\
    `establish` varchar(10) NOT NULL COMMENT '설립구분',\
    `status` varchar(10) NOT NULL COMMENT '상태',\
    `campus` varchar(10) NOT NULL COMMENT '캠퍼스',\
    `campus_name` varchar(50) NOT NULL COMMENT '캠퍼스명',\
    `locale` varchar(10) NOT NULL COMMENT '지역',\
    `homepage` varchar(50) NOT NULL COMMENT '학교 홈페이지',\
    `tel` varchar(20) NOT NULL COMMENT '전화번호',\
    `fax` varchar(20) NOT NULL COMMENT '팩스번호',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='대학교정보';"
table_dic['대학교정보'] = 'university'    

# 중도탈락 현황 테이블(수동으로 통합 변경된 테이블)
table_dic['dropout_stat'] = \
    "CREATE TABLE `dropout_stat` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `stud_tot` int(5) DEFAULT 0 COMMENT '재적_학생수',\
    `stud_sum` int(5) DEFAULT 0 COMMENT '재적_중도탈락_합계',\
    `stud_unregist` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미등록',\
    `stud_unreturn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미복학',\
    `stud_drop` int(5) DEFAULT 0 COMMENT '재적_중도탈락_자퇴',\
    `stud_warn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학사경고',\
    `stud_act` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학생활동',\
    `stud_expulsion` int(5) DEFAULT 0 COMMENT '재적_중도탈락_유급제적',\
    `stud_attendyear` int(5) DEFAULT 0 COMMENT '재적_중도탈락_재학연한초과',\
    `stud_etc` int(5) DEFAULT 0 COMMENT '재적_중도탈락_기타',\
    `stud_rate` float(5,2) DEFAULT 0.00 COMMENT '재적_중도탈락_비율',\
    `fresh_tot` int(5) DEFAULT 0 COMMENT '신입_학생수',\
    `fresh_sum` int(5) DEFAULT 0 COMMENT '신입_중도탈락_합계',\
    `fresh_unregist` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미등록',\
    `fresh_unreturn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미복학',\
    `fresh_drop` int(5) DEFAULT 0 COMMENT '신입_중도탈락_자퇴',\
    `fresh_warn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학사경고',\
    `fresh_act` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학생활동',\
    `fresh_expulsion` int(5) DEFAULT 0 COMMENT '신입_중도탈락_유급제적',\
    `fresh_attendyear` int(5) DEFAULT 0 COMMENT '신입_중도탈락_재학연한초과',\
    `fresh_etc` int(5) DEFAULT 0 COMMENT '신입_중도탈락_기타',\
    `fresh_rate` float(5,2) DEFAULT 0.00 COMMENT '신입_중도탈락_비율',\
    `foreign_tot` int(5) DEFAULT 0 COMMENT '외국인_학생수',\
    `foreign_sum` int(5) DEFAULT 0 COMMENT '외국인_중도탈락_합계',\
    `foreign_degree` int(5) DEFAULT 0 COMMENT '외국인_학위과정(대학)',\
    `foreign_common` int(5) DEFAULT 0 COMMENT '외국인_교육과정공동운영생',\
    `foreign_lang` int(5) DEFAULT 0 COMMENT '외국인_어학연수생',\
    `foreign_exchange` int(5) DEFAULT 0 COMMENT '외국인_교환학생',\
    `foreign_visit` int(5) DEFAULT 0 COMMENT '외국인_방문학생',\
    `foreign_etc` int(5) DEFAULT 0 COMMENT '외국인_기타연수생',\
    `foreign_rate` float(5,2) DEFAULT 0.00 COMMENT '외국인_중도탈락_비율',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `dropout_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='중도탈락 학생 현황';"
table_dic['중도탈락 학생 현황(통합)'] = 'dropout_stat'        
    
# 2023년_ [2-가-1. 전공과목 성적 분포_] 테이블
#################################################
# 테이블명 : major_grades | 전공과목 성적 분포
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL    | semester | stud_tot                 | perfect_rating  | stud_A+   | rate_A+ | stud_A0 | rate_A0  | stud_B+  | rate_B+ | stud_B0 | rate_B0  | stud_C+ | rate_C+ | stud_C0 | rate_C0   | stud_D+  | rate_D+ | stud_D0 | rate_D0  | stud_F   | rate_F
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 학기      | 전공과목 성적인정 총학생수 | 만점평점         | A+ 학생수 | A+ 비율 | A0 학생수 | A0 비율 | B+ 학생수 | B+ 비율 | B0 학생수 | B0 비율 | C+ 학생수 | C+ 비율 | C0 학생수 | C0 비율 | D+ 학생수 | D+ 비율 | D0 학생수 | D0 비율 | F 학생수 | F 비율 
#################################################
table_dic['major_grades'] = \
    "CREATE TABLE `major_grades` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `semester` varchar(10) NOT NULL COMMENT '학기',\
    `stud_tot` int(5) DEFAULT 0 COMMENT '전공과목 성적인정 총학생수',\
    `perfect_rating` float(5,2) DEFAULT 0 COMMENT '만점평점',\
    `stud_Ap` int(5) DEFAULT 0 COMMENT 'A+ 학생수',\
    `rate_Ap` float(5,2) DEFAULT 0.00 COMMENT 'A+ 비율',\
    `stud_A0` int(5) DEFAULT 0 COMMENT 'A0 학생수',\
    `rate_A0` float(5,2) DEFAULT 0.00 COMMENT 'A0 비율',\
    `stud_Bp` int(5) DEFAULT 0 COMMENT 'B+ 학생수',\
    `rate_Bp` float(5,2) DEFAULT 0.00 COMMENT 'B+ 비율',\
    `stud_B0` int(5) DEFAULT 0 COMMENT 'B0 학생수',\
    `rate_B0` float(5,2) DEFAULT 0.00 COMMENT 'B0 비율',\
    `stud_Cp` int(5) DEFAULT 0 COMMENT 'C+ 학생수',\
    `rate_Cp` float(5,2) DEFAULT 0.00 COMMENT 'C+ 비율',\
    `stud_C0` int(5) DEFAULT 0 COMMENT 'C0 학생수',\
    `rate_C0` float(5,2) DEFAULT 0.00 COMMENT 'C0 비율',\
    `stud_Dp` int(5) DEFAULT 0 COMMENT 'D+ 학생수',\
    `rate_Dp` float(5,2) DEFAULT 0.00 COMMENT 'D+ 비율',\
    `stud_D0` int(5) DEFAULT 0 COMMENT 'D0 학생수',\
    `rate_D0` float(5,2) DEFAULT 0.00 COMMENT 'D0 비율',\
    `stud_F` int(5) DEFAULT 0 COMMENT 'F 학생수',\
    `rate_F` float(5,2) DEFAULT 0.00 COMMENT 'F 비율',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`, `semester`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `major_grades_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='전공과목 성적 분포';"
table_dic['전공과목 성적 분포'] = 'major_grades'


# 2023년_ [2-가-2. 교양과목 성적 분포_] 테이블
#################################################
# 테이블명 : liberal_arts_grades | 교양과목 성적 분포
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL    | semester | stud_tot                 | perfect_rating  | stud_A+   | rate_A+ | stud_A0 | rate_A0  | stud_B+  | rate_B+ | stud_B0 | rate_B0  | stud_C+ | rate_C+ | stud_C0 | rate_C0   | stud_D+  | rate_D+ | stud_D0 | rate_D0  | stud_F   | rate_F
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 학기      | 교양과목 성적인정 총학생수 | 만점평점         | A+ 학생수 | A+ 비율 | A0 학생수 | A0 비율 | B+ 학생수 | B+ 비율 | B0 학생수 | B0 비율 | C+ 학생수 | C+ 비율 | C0 학생수 | C0 비율 | D+ 학생수 | D+ 비율 | D0 학생수 | D0 비율 | F 학생수 | F 비율 
#################################################
table_dic['liberal_arts_grades'] = \
    "CREATE TABLE `liberal_arts_grades` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `semester` varchar(10) NOT NULL COMMENT '학기',\
    `stud_tot` int(5) DEFAULT 0 COMMENT '교양과목 성적인정 총학생수',\
    `perfect_rating` float(5,2) DEFAULT 0 COMMENT '만점평점',\
    `stud_Ap` int(5) DEFAULT 0 COMMENT 'A+ 학생수',\
    `rate_Ap` float(5,2) DEFAULT 0.00 COMMENT 'A+ 비율',\
    `stud_A0` int(5) DEFAULT 0 COMMENT 'A0 학생수',\
    `rate_A0` float(5,2) DEFAULT 0.00 COMMENT 'A0 비율',\
    `stud_Bp` int(5) DEFAULT 0 COMMENT 'B+ 학생수',\
    `rate_Bp` float(5,2) DEFAULT 0.00 COMMENT 'B+ 비율',\
    `stud_B0` int(5) DEFAULT 0 COMMENT 'B0 학생수',\
    `rate_B0` float(5,2) DEFAULT 0.00 COMMENT 'B0 비율',\
    `stud_Cp` int(5) DEFAULT 0 COMMENT 'C+ 학생수',\
    `rate_Cp` float(5,2) DEFAULT 0.00 COMMENT 'C+ 비율',\
    `stud_C0` int(5) DEFAULT 0 COMMENT 'C0 학생수',\
    `rate_C0` float(5,2) DEFAULT 0.00 COMMENT 'C0 비율',\
    `stud_Dp` int(5) DEFAULT 0 COMMENT 'D+ 학생수',\
    `rate_Dp` float(5,2) DEFAULT 0.00 COMMENT 'D+ 비율',\
    `stud_D0` int(5) DEFAULT 0 COMMENT 'D0 학생수',\
    `rate_D0` float(5,2) DEFAULT 0.00 COMMENT 'D0 비율',\
    `stud_F` int(5) DEFAULT 0 COMMENT 'F 학생수',\
    `rate_F` float(5,2) DEFAULT 0.00 COMMENT 'F 비율',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`, `semester`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `liberal_arts_grades_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='교양과목 성적 분포';"
table_dic['교양과목 성적 분포'] = 'liberal_arts_grades'


# 2023년_ [2-가-3. 교직과목 성적 분포_] 테이블
#################################################
# 테이블명 : teaching_profession_grades | 교직과목 성적 분포
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL    | semester | stud_tot                 | perfect_rating  | stud_A+   | rate_A+ | stud_A0 | rate_A0  | stud_B+  | rate_B+ | stud_B0 | rate_B0  | stud_C+ | rate_C+ | stud_C0 | rate_C0   | stud_D+  | rate_D+ | stud_D0 | rate_D0  | stud_F   | rate_F
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 학기      | 교직과목 성적인정 총학생수 | 만점평점         | A+ 학생수 | A+ 비율 | A0 학생수 | A0 비율 | B+ 학생수 | B+ 비율 | B0 학생수 | B0 비율 | C+ 학생수 | C+ 비율 | C0 학생수 | C0 비율 | D+ 학생수 | D+ 비율 | D0 학생수 | D0 비율 | F 학생수 | F 비율 
#################################################
table_dic['teaching_profession_grades'] = \
    "CREATE TABLE `teaching_profession_grades` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `semester` varchar(10) NOT NULL COMMENT '학기',\
    `stud_tot` int(5) DEFAULT 0 COMMENT '교직과목 성적인정 총학생수',\
    `perfect_rating` float(5,2) DEFAULT 0 COMMENT '만점평점',\
    `stud_Ap` int(5) DEFAULT 0 COMMENT 'A+ 학생수',\
    `rate_Ap` float(5,2) DEFAULT 0.00 COMMENT 'A+ 비율',\
    `stud_A0` int(5) DEFAULT 0 COMMENT 'A0 학생수',\
    `rate_A0` float(5,2) DEFAULT 0.00 COMMENT 'A0 비율',\
    `stud_Bp` int(5) DEFAULT 0 COMMENT 'B+ 학생수',\
    `rate_Bp` float(5,2) DEFAULT 0.00 COMMENT 'B+ 비율',\
    `stud_B0` int(5) DEFAULT 0 COMMENT 'B0 학생수',\
    `rate_B0` float(5,2) DEFAULT 0.00 COMMENT 'B0 비율',\
    `stud_Cp` int(5) DEFAULT 0 COMMENT 'C+ 학생수',\
    `rate_Cp` float(5,2) DEFAULT 0.00 COMMENT 'C+ 비율',\
    `stud_C0` int(5) DEFAULT 0 COMMENT 'C0 학생수',\
    `rate_C0` float(5,2) DEFAULT 0.00 COMMENT 'C0 비율',\
    `stud_Dp` int(5) DEFAULT 0 COMMENT 'D+ 학생수',\
    `rate_Dp` float(5,2) DEFAULT 0.00 COMMENT 'D+ 비율',\
    `stud_D0` int(5) DEFAULT 0 COMMENT 'D0 학생수',\
    `rate_D0` float(5,2) DEFAULT 0.00 COMMENT 'D0 비율',\
    `stud_F` int(5) DEFAULT 0 COMMENT 'F 학생수',\
    `rate_F` float(5,2) DEFAULT 0.00 COMMENT 'F 비율',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`, `semester`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `teaching_profession_grades_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='교직과목 성적 분포';"
table_dic['교직과목 성적 분포'] = 'liberal_arts_grades'



# 2023년_ [2-가-4. 졸업생의 졸업성적 분포_]
#################################################
# 테이블명 : graduate_grades | 졸업생의 졸업성적 분포
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | send_tot        | send_0_3            | send_3_6           | send_6_12           | send_12          | receive_tot        | receive_0_3            | receive_3_6           | receive_6_12           | receive_12
#################################################
table_dic['graduate_grades'] = \
    "CREATE TABLE `graduate_grades` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `graduate_tot` varchar(50) COMMENT '졸업자 수',\
    `department_avg_grade` varchar(50) COMMENT '학과(전공) 별 졸업평점 평균',\
    `perfect_rating` varchar(50) COMMENT '만점평점',\
    `department_avg_rate` varchar(50) COMMENT '학과(전공) 별 졸업 백분율 점수 평균',\
    `grade_100_95` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_100_95',\
    `rate_100_95` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_100_95',\
    `grade_95_90` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_95_90',\
    `rate_95_90` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_95_90',\
    `grade_90_85` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_90_85',\
    `rate_90_85` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_90_85',\
    `grade_85_80` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_85_80',\
    `rate_85_80` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_85_80',\
    `grade_80_75` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_80_75',\
    `rate_80_75` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_80_75',\
    `grade_75_70` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_75_70',\
    `rate_75_70` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_75_70',\
    `grade_70_65` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_70_65',\
    `rate_70_65` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_70_65',\
    `grade_65_60` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_65_60',\
    `rate_65_60` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_65_60',\
    `grade_60` int(5) DEFAULT 0 COMMENT '평점(점수구간)별 학생수(대학에서 정한 학업성적 등급)_60미만',\
    `rate_60` float(5, 2) DEFAULT 0.00 COMMENT '평점(점수구간)별 비율(대학에서 정한 학업성적 등급)_60미만',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `graduate_grades_exchange_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='졸업생의 졸업성적 분포';"
table_dic['졸업생의 졸업성적 분포'] = 'graduate_grades'




# 2023년_ [3-나. 전임 입학사정관 현황_]
#################################################
# 테이블명 : admissions_officer | 전임 입학사정관 현황
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | completed_stud_dept | completed_stud_human_social | completed_stud_natural_science | completed_stud_engineering | completed_stud_medicine | support_amount_stud_tot | support_amount_prototype | support_amount_education |
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 이수학생수_해당학과 | 이수학생수_타학과_인문사회 | 이수학생수_타학과_자연과학 | 이수학생수_타학과_공학 | 이수학생수_타학과_의학 | 이수학생수_타학과_예체능 | 지원금수령_학생수(명) | 지원금수령_금액_시제품제작비 | 지원금수령_금액_교육프로그램지원비
#################################################
table_dic['admissions_officer'] = \
    "CREATE TABLE `admissions_officer` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `employ_office_tot` int(5) DEFAULT 0 COMMENT '채용사정관_총인원',\
    `employ_office_ft_tot` int(5) DEFAULT 0 COMMENT '채용사정관_정규직인원',\
    `employ_office_ft_rate` float(5, 2) DEFAULT 0.00 COMMENT '채용사정관_정규직비율',\
    `professor_office_tot` int(5) DEFAULT 0 COMMENT '교수사정관_총인원',\
    `professor_office_ft_tot` int(5) DEFAULT 0 COMMENT '교수사정관_정규직인원',\
    `professor_office_ft_rate` float(5, 2) DEFAULT 0.00 COMMENT '교수사정관_정규직비율',\
    `transition_office_tot` int(5) DEFAULT 0 COMMENT '전환사정관_총인원',\
    `transition_office_ft_tot` int(5) DEFAULT 0 COMMENT '전환사정관_정규직인원',\
    `transition_office_ft_rate` float(5, 2) DEFAULT 0.00 COMMENT '전환사정관_정규직비율',\
    `sub_office_tot` int(5) DEFAULT 0 COMMENT '소계_사정관 총인원',\
    `sub_office_ft_tot` int(5) DEFAULT 0 COMMENT '소계_사정관_정규직인원',\
    `sub_office_ft_rate` float(5, 2) DEFAULT 0.00 COMMENT '소계_사정관 정규직비율',\
    `appointment_tot` int(5) DEFAULT 0 COMMENT '위촉사정관_총인원',\
    `total_tot` int(5) DEFAULT 0 COMMENT '총계_총인원',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `admissions_officer_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='전임 입학사정관 현황';"
table_dic['전임 입학사정관 현황'] = 'admissions_officer'


# 2023년_ [4-나. 기회 균형 선발 결과_]
#################################################
# 테이블명 : opportunity_select_results | 기회 균형 선발 결과
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | completed_stud_dept | completed_stud_human_social | completed_stud_natural_science | completed_stud_engineering | completed_stud_medicine | support_amount_stud_tot | support_amount_prototype | support_amount_education |
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 이수학생수_해당학과 | 이수학생수_타학과_인문사회 | 이수학생수_타학과_자연과학 | 이수학생수_타학과_공학 | 이수학생수_타학과_의학 | 이수학생수_타학과_예체능 | 지원금수령_학생수(명) | 지원금수령_금액_시제품제작비 | 지원금수령_금액_교육프로그램지원비
#################################################
table_dic['opportunity_select_results'] = \
    "CREATE TABLE `opportunity_select_results` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `admission_tot` int(5) DEFAULT 0 COMMENT '전체 총 입학자',\
    `opportunity_tot` int(5) DEFAULT 0 COMMENT '기회균형 입학 합계(정원외소계+정원내소계)',\
    `out_recruitment_subtot` int(5) DEFAULT 0 COMMENT '정원외_소계',\
    `out_recruitment_basic_livelihood` int(5) DEFAULT 0 COMMENT '정원외_기초생활수급자',\
    `out_recruitment_second_lower` int(5) DEFAULT 0 COMMENT '정원외_차상위계층',\
    `out_recruitment_single_parent` int(5) DEFAULT 0 COMMENT '정원외_한부모가족 지원대상자',\
    `out_recruitment_special_school` int(5) DEFAULT 0 COMMENT '정원외_특성화고 졸업자',\
    `out_recruitment_special_school_graduate` int(5) DEFAULT 0 COMMENT '정원외_특성화고 등을 졸업한 재직자',\
    `out_recruitment_rural_student` int(5) DEFAULT 0 COMMENT '정원외_농어촌학생',\
    `out_recruitment_north_defector` int(5) DEFAULT 0 COMMENT '정원외_북한 이탈주민',\
    `out_recruitment_disabled` int(5) DEFAULT 0 COMMENT '정원외_장애인 등 대상자',\
    `out_recruitment_west_sea` int(5) DEFAULT 0 COMMENT '정원외_서해 5도 학생',\
    `out_recruitment_danwon_school` int(5) DEFAULT 0 COMMENT '정원외_단원고',\
    `in_recruitment_subtot` int(5) DEFAULT 0 COMMENT '정원내_소계',\
    `in_recruitment_opportunity` int(5) DEFAULT 0 COMMENT '정원내_고른기회 대상자',\
    `in_recruitment_univ_standard` int(6) DEFAULT 0 COMMENT '정원내_대학 독자적 기준',\
    `opportunity_rate` float(5, 2) DEFAULT 0.00 COMMENT '기회균형 선발 비율(합계/총입학자*100)',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `opportunity_select_results_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='기회 균형 선발 결과';"
table_dic['기회 균형 선발 결과'] = 'opportunity_select_results'






# 2023년_ [12-라-2. 외국대학과 학점 교류 현황_]
#################################################
# 테이블명 : major_grades | 전공과목 성적 분포
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | send_tot        | send_0_3            | send_3_6           | send_6_12           | send_12          | receive_tot        | receive_0_3            | receive_3_6           | receive_6_12           | receive_12
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 파견인원(자교->타교) | 파견기간별 인원(0<3개월) | 파견기간별 인원(3<6개월) | 파견기간별 인원(6<12개월) | 파견기간별 인원(12개월 이상) | 유치인원(타교->자교) | 유치기간별 인원(0<3개월) | 유치기간별 인원(3<6개월) | 유치기간별 인원(6<12개월) | 유치기간별 인원(12개월 이상) |
#################################################
table_dic['foreign_univ_grade_exchange'] = \
    "CREATE TABLE `foreign_univ_grade_exchange` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `send_tot` int(5) DEFAULT 0 COMMENT '파견인원(자교->타교)',\
    `send_0_3` int(5) DEFAULT 0 COMMENT '파견기간별 인원(0<3개월)',\
    `send_3_6` int(5) DEFAULT 0 COMMENT '파견기간별 인원(3<6개월)',\
    `send_6_12` int(5) DEFAULT 0 COMMENT '파견기간별 인원(6<12개월)',\
    `send_12` int(5) DEFAULT 0 COMMENT '파견기간별 인원(12개월 이상)',\
    `receive_tot` int(5) DEFAULT 0 COMMENT '유치인원(타교->자교)',\
    `receive_0_3` int(5) DEFAULT 0 COMMENT '유치기간별 인원(0<3개월)',\
    `receive_3_6` int(5) DEFAULT 0 COMMENT '유치기간별 인원(3<6개월)',\
    `receive_6_12` int(5) DEFAULT 0 COMMENT '유치기간별 인원(6<12개월)',\
    `sreceive_12` int(5) DEFAULT 0 COMMENT '유치기간별 인원(12개월 이상)',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `foreign_univ_grade_exchange_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='외국대학과 학점 교류 현황';"
table_dic['외국대학과 학점 교류 현황'] = 'foreign_univ_grade_exchange'


# 2023년_ [12-카-1. 현장실습 운영 현황_]
#################################################
# 테이블명 : field_training | 현장실습 운영 현황
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | ft_4w_tot | ft_4w_insure_wound | ft_4w_insure_industrial | ft_4w_fee_75p | ft_4w_fee_100p | ft_4w_home_25p_under | ft_4w_home_25p_above
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 실습기간_4주_이수학생수 | 실습기간_4주_보험가입_상해보험 | 실습기간_4주_보험가입_산재보험 | 실습기간_4주_실습지원비_75%이상 | 실습기간_4주_실습지원비_100%이상 | 실습기간_4주_재택실습이용학생_기간25%이하 | 실습기간_4주_재택실습이용학생_기간25%초과 
#                                                          | 실습기간_8주_이수학생수 | 실습기간_8주_보험가입_상해보험 | 실습기간_8주_보험가입_산재보험 | 실습기간_8주_실습지원비_75%이상 | 실습기간_8주_실습지원비_100%이상 | 실습기간_8주_재택실습이용학생_기간25%이하 | 실습기간_8주_재택실습이용학생_기간25%초과 
#                                                          | 실습기간_12주_이수학생수 | 실습기간_12주_보험가입_상해보험 | 실습기간_12주_보험가입_산재보험 | 실습기간_12주_실습지원비_75%이상 | 실습기간_12주_실습지원비_100%이상 | 실습기간_12주_재택실습이용학생_기간25%이하 | 실습기간_12주_재택실습이용학생_기간25%초과 
#################################################
table_dic['field_training'] = \
    "CREATE TABLE `field_training` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `ft_4w_tot` int(5) DEFAULT 0 COMMENT '실습기간_4주_이수학생수',\
    `ft_4w_insure_wound` int(5) DEFAULT 0 COMMENT '실습기간_4주_보험가입_상해보험',\
    `ft_4w_insure_industrial` int(5) DEFAULT 0 COMMENT '실습기간_4주_보험가입_산재보험',\
    `ft_4w_fee_75p` int(5) DEFAULT 0 COMMENT '실습기간_4주_실습지원비_75%이상',\
    `ft_4w_fee_100p` int(5) DEFAULT 0 COMMENT '실습기간_4주_실습지원비_100%이상',\
    `ft_4w_home_25p_under` int(5) DEFAULT 0 COMMENT '실습기간_4주_재택실습이용학생_기간25%이하',\
    `ft_4w_home_25p_above` int(5) DEFAULT 0 COMMENT '실습기간_4주_재택실습이용학생_기간25%초과',\
    `ft_8w_tot` int(5) DEFAULT 0 COMMENT '실습기간_8주_이수학생수',\
    `ft_8w_insure_wound` int(5) DEFAULT 0 COMMENT '실습기간_8주_보험가입_상해보험',\
    `ft_8w_insure_industrial` int(5) DEFAULT 0 COMMENT '실습기간_8주_보험가입_산재보험',\
    `ft_8w_fee_75p` int(5) DEFAULT 0 COMMENT '실습기간_8주_실습지원비_75%이상',\
    `ft_8w_fee_100p` int(5) DEFAULT 0 COMMENT '실습기간_8주_실습지원비_100%이상',\
    `ft_8w_home_25p_under` int(5) DEFAULT 0 COMMENT '실습기간_8주_재택실습이용학생_기간25%이하',\
    `ft_8w_home_25p_above` int(5) DEFAULT 0 COMMENT '실습기간_8주_재택실습이용학생_기간25%초과',\
    `ft_12w_tot` int(5) DEFAULT 0 COMMENT '실습기간_12주_이수학생수',\
    `ft_12w_insure_wound` int(5) DEFAULT 0 COMMENT '실습기간_12주_보험가입_상해보험',\
    `ft_12w_insure_industrial` int(5) DEFAULT 0 COMMENT '실습기간_12주_보험가입_산재보험',\
    `ft_12w_fee_75p` int(5) DEFAULT 0 COMMENT '실습기간_12주_실습지원비_75%이상',\
    `ft_12w_fee_100p` int(5) DEFAULT 0 COMMENT '실습기간_12주_실습지원비_100%이상',\
    `ft_12w_home_25p_under` int(5) DEFAULT 0 COMMENT '실습기간_12주_재택실습이용학생_기간25%이하',\
    `ft_12w_home_25p_above` int(5) DEFAULT 0 COMMENT '실습기간_12주_재택실습이용학생_기간25%초과',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `field_training_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='신입생 충원 현황';"
table_dic['현장실습 운영 현황'] = 'field_training'


# 2023년_ [12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_]
#################################################
# 테이블명 : capstone_design | 캡스톤 디자인(창의적 설계) 운영 현황
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | completed_stud_dept | completed_stud_human_social | completed_stud_natural_science | completed_stud_engineering | completed_stud_medicine | completed_stud_art_sports | support_amount_stud_tot | support_amount_prototype | support_amount_education |
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 이수학생수_해당학과 | 이수학생수_타학과_인문사회 | 이수학생수_타학과_자연과학 | 이수학생수_타학과_공학 | 이수학생수_타학과_의학 | 이수학생수_타학과_예체능 | 지원금수령_학생수(명) | 지원금수령_금액_시제품제작비 | 지원금수령_금액_교육프로그램지원비
#################################################
table_dic['capstone_design'] = \
    "CREATE TABLE `capstone_design` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `completed_stud_dept` int(5) DEFAULT 0 COMMENT '이수학생수_해당학과',\
    `completed_stud_human_social` int(5) DEFAULT 0 COMMENT '이수학생수_타학과_인문사회',\
    `completed_stud_natural_science` int(5) DEFAULT 0 COMMENT '이수학생수_타학과_자연과학',\
    `completed_stud_engineering` int(5) DEFAULT 0 COMMENT '이수학생수_타학과_공학',\
    `completed_stud_medicine` int(5) DEFAULT 0 COMMENT '이수학생수_타학과_의학',\
    `completed_stud_art_sports` int(5) DEFAULT 0 COMMENT '이수학생수_타학과_예체능',\
    `support_amount_stud_tot` int(5) DEFAULT 0 COMMENT '지원금수령_학생수(명)',\
    `support_amount_prototype` int(5) DEFAULT 0 COMMENT '지원금수령_금액_시제품제작비',\
    `support_amount_education` int(5) DEFAULT 0 COMMENT '지원금수령_금액_교육프로그램지원비',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `capstone_design_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='캡스톤 디자인(창의적 설계) 운영 현황';"
table_dic['캡스톤 디자인(창의적 설계) 운영 현황'] = 'capstone_design'



# 2023년_ [4-라-2. 편입학 선발 결과_]
#################################################
# 테이블명 : transfer_student | 2023년_ [4-라-2. 편입학 선발 결과_]
# 모집인원_정원내_일반편입 | 모집인원_정원내_기타 | 모집인원_정원외_학사편입 | 모집인원_정원외_특별편입학 | 모집인원_정원외_기타 | 지원자_정원내_일반편입 | 지원자_정원내_기타 | 지원자_정원외_학사편입 | 지원자_정원외_특별편입학 | 지원자_정원외_기타 | 등록인원_정원내_일반편입 | 등록인원_정원내_기타 | 등록인원_정원외_학사편입 | 
#################################################
table_dic['transfer_student'] = \
    "CREATE TABLE `transfer_student` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `recruitment_in_general` int(5) DEFAULT 0 COMMENT '모집인원_정원내_일반편입',\
    `recruitment_in_etc` int(5) DEFAULT 0 COMMENT '모집인원_정원내_기타',\
    `recruitment_out_bachelor` int(5) DEFAULT 0 COMMENT '모집인원_정원외_학사편입',\
    `recruitment_out_special` int(5) DEFAULT 0 COMMENT '모집인원_정원외_특별편입학',\
    `recruitment_out_etc` int(5) DEFAULT 0 COMMENT '모집인원_정원외_기타',\
    `applicant_in_general` int(5) DEFAULT 0 COMMENT '지원자_정원내_일반편입',\
    `applicant_in_etc` int(5) DEFAULT 0 COMMENT '지원자_정원내_기타',\
    `applicant_out_bachelor` int(5) DEFAULT 0 COMMENT '지원자_정원외_학사편입',\
    `applicant_out_special` int(5) DEFAULT 0 COMMENT '지원자_정원외_특별편입학',\
    `applicant_out_etc` int(5) DEFAULT 0 COMMENT '지원자_정원외_기타',\
    `registered_in_general` int(5) DEFAULT 0 COMMENT '등록인원_정원내_일반편입',\
    `registered_in_etc` int(5) DEFAULT 0 COMMENT '등록인원_정원내_기타',\
    `registered_out_bachelor` int(5) DEFAULT 0 COMMENT '등록인원_정원외_학사편입',\
    `registered_out_special` int(5) DEFAULT 0 COMMENT '등록인원_정원외_특별편입학',\
    `registered_out_etc` int(5) DEFAULT 0 COMMENT '등록인원_정원외_기타',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `transfer_student_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='편입학 선발 결과';"
table_dic['편입학 선발 결과'] = 'transfer_student'


# 2023년_ [4-마. 재적 학생 현황_]
#################################################
# 테이블명 : enrolled_student | 2023년_ [4-마. 재적 학생 현황_]
# 학생정원 | 재학생_계_정원내 | 재학생_계_정원외 | 재학생_남_정원내 | 재학생_남_정원외 | 재학생_여_정원내 | 재학생_여_정원외 | 휴학생_계_정원내 | 휴학생_계_정원외 | 휴학생_남_정원내 | 휴학생_남_정원외 | 휴학생_여_정원내 | 휴학생_여_정원외 | 
#################################################
table_dic['enrolled_student'] = \
    "CREATE TABLE `enrolled_student` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `personnel_student_tot` int(5) DEFAULT 0 COMMENT '학생정원',\
    `enrolled_sum_in` int(5) DEFAULT 0 COMMENT '재학생_계_정원내',\
    `enrolled_sum_out` int(5) DEFAULT 0 COMMENT '재학생_계_정원외',\
    `enrolled_male_in` int(5) DEFAULT 0 COMMENT '재학생_남_정원내',\
    `enrolled_male_out` int(5) DEFAULT 0 COMMENT '재학생_남_정원외',\
    `enrolled_female_in` int(5) DEFAULT 0 COMMENT '재학생_여_정원내',\
    `enrolled_female_out` int(5) DEFAULT 0 COMMENT '재학생_여_정원외',\
    `absence_sum_in` int(5) DEFAULT 0 COMMENT '휴학생_계_정원내',\
    `absence_sum_out` int(5) DEFAULT 0 COMMENT '휴학생_계_정원외',\
    `absence_male_in` int(5) DEFAULT 0 COMMENT '휴학생_남_정원내',\
    `absence_male_out` int(5) DEFAULT 0 COMMENT '휴학생_남_정원외',\
    `absence_female_in` int(5) DEFAULT 0 COMMENT '휴학생_여_정원내',\
    `absence_female_out` int(5) DEFAULT 0 COMMENT '휴학생_여_정원외',\
    `suspension_sum_in` int(5) DEFAULT 0 COMMENT '학사학위취득유예학생_계_정원내',\
    `suspension_sum_out` int(5) DEFAULT 0 COMMENT '학사학위취득유예학생_계_정원외',\
    `suspension_male_in` int(6) DEFAULT 0 COMMENT '학사학위취득유예학생_남_정원내',\
    `suspension_male_out` int(7) DEFAULT 0 COMMENT '학사학위취득유예학생_남_정원외',\
    `suspension_female_in` int(8) DEFAULT 0 COMMENT '학사학위취득유예학생_여_정원내',\
    `suspension_female_out` int(9) DEFAULT 0 COMMENT '학사학위취득유예학생_여_정원외',\
    `attend_sum_in` int(10) DEFAULT 0 COMMENT '재적학생_계_정원내',\
    `attend_sum_out` int(11) DEFAULT 0 COMMENT '재적학생_계_정원외',\
    `attend_male_in` int(12) DEFAULT 0 COMMENT '재적학생_남_정원내',\
    `attend_male_out` int(13) DEFAULT 0 COMMENT '재적학생_남_정원외',\
    `attend_female_in` int(14) DEFAULT 0 COMMENT '재적학생_여_정원내',\
    `attend_female_out` int(15) DEFAULT 0 COMMENT '재적학생_여_정원외',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `enrolled_student_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='재적 학생 현황';"
table_dic['재적 학생 현황'] = 'enrolled_student'


# 2023년_ [4-바-1. 외국학생 현황_]
#################################################
# 테이블명 : foreign_student | 2023년_ [4-바-1. 외국학생 현황_]
# 외국학생 총계 | 학위과정_소계 | 학위과정_인문사회계열 | 학위과정_자연과학계열 | 학위과정_공학계열 | 학위과정_예체능계열 | 학위과정_의학계열 | 교육과정 공동운영생 | 연수과정_소계 | 연수과정_어학연수생 | 연수과정_교환학생 | 연수과정_방문학생 | 연수과정_기타연수생 | 
#################################################
table_dic['foreign_student'] = \
    "CREATE TABLE `foreign_student` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `foreign_tot` int(5) DEFAULT 0 COMMENT '외국학생 총계',\
    `degree_course_sum` int(5) DEFAULT 0 COMMENT '학위과정_소계',\
    `degree_course_social_sciences` int(5) DEFAULT 0 COMMENT '학위과정_인문사회계열',\
    `degree_course_natural_science` int(5) DEFAULT 0 COMMENT '학위과정_자연과학계열',\
    `degree_course_engineering` int(5) DEFAULT 0 COMMENT '학위과정_공학계열',\
    `degree_course_arts_physical` int(5) DEFAULT 0 COMMENT '학위과정_예체능계열',\
    `degree_course_medical` int(5) DEFAULT 0 COMMENT '학위과정_의학계열',\
    `joint_operation` int(5) DEFAULT 0 COMMENT '교육과정 공동운영생',\
    `training_course_sum` int(5) DEFAULT 0 COMMENT '연수과정_소계',\
    `training_course_language` int(5) DEFAULT 0 COMMENT '연수과정_어학연수생',\
    `training_course_exchange` int(5) DEFAULT 0 COMMENT '연수과정_교환학생',\
    `training_course_visiting` int(5) DEFAULT 0 COMMENT '연수과정_방문학생',\
    `training_course_etc` int(5) DEFAULT 0 COMMENT '연수과정_기타연수생',\
    `language_skill_sum` int(5) DEFAULT 0 COMMENT '언어능력_소계',\
    `language_skill_topik4` int(5) DEFAULT 0 COMMENT '언어능력_TOPIK 4급(예체능3급)이상',\
    `language_skill_toefl530` int(5) DEFAULT 0 COMMENT '언어능력_영어트랙TOEFL 530이상',\
    `language_skill_anglosphere` int(5) DEFAULT 0 COMMENT '언어능력_영어권 외국인유학생',\
    `language_skill_rate` float(5, 2) DEFAULT 0.00 COMMENT '언어능력충족 학생비율(언어능력_소계/총계)',\
    `dormitory_degree_sum` int(5) DEFAULT 0 COMMENT '기숙사수용_학위과정_소계',\
    `dormitory_degree_accept` int(5) DEFAULT 0 COMMENT '기숙사수용_학위과정_수용',\
    `dormitory_degree_notaccept` int(5) DEFAULT 0 COMMENT '기숙사수용_학위과정_미수용',\
    `dormitory_none_degree_sum` int(5) DEFAULT 0 COMMENT '기숙사수용_비학위과정_소계',\
    `dormitory_none_degree_accept` int(5) DEFAULT 0 COMMENT '기숙사수용_비학위과정_수용',\
    `dormitory_none_degree_notaccept` int(5) DEFAULT 0 COMMENT '기숙사수용_비학위과정_미수용',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `foreign_student_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='외국학생 현황';"
table_dic['외국학생 현황'] = 'foreign_student'



# 2023년_ [4-바-2. 외국학생 중도탈락 현황_]
#################################################
# 테이블명 : foreign_dropout_status | 2023년_ [4-바-2. 외국학생 중도탈락 현황_]
# 외국재적학생 총계 | 외국재적학생_학위과정(대학) | 외국재적학생_교육과정공동운영생 | 외국재적학생_어학연수생 | 외국재적학생_교환학생 | 외국재적학생_방문학생 | 외국재적학생_기타연수생 | 중도탈락_총계 | 중도탈락_학위과정(대학) | 중도탈락_교육과정공동운영생 | 중도탈락_어학연수생 | 중도탈락_교환학생 | 중도탈락_방문학생 | 
#################################################
table_dic['foreign_dropout_status'] = \
    "CREATE TABLE `foreign_dropout_status` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `foreign_tot` int(5) DEFAULT 0 COMMENT '외국재적학생 총계',\
    `foreign_degree_course` int(5) DEFAULT 0 COMMENT '외국재적학생_학위과정(대학)',\
    `foreign_joint_operation` int(5) DEFAULT 0 COMMENT '외국재적학생_교육과정공동운영생',\
    `foreign_course_language` int(5) DEFAULT 0 COMMENT '외국재적학생_어학연수생',\
    `foreign_course_exchange` int(5) DEFAULT 0 COMMENT '외국재적학생_교환학생',\
    `foreign_course_visiting` int(5) DEFAULT 0 COMMENT '외국재적학생_방문학생',\
    `foreign_etc` int(5) DEFAULT 0 COMMENT '외국재적학생_기타연수생',\
    `dropout_tot` int(5) DEFAULT 0 COMMENT '중도탈락_총계',\
    `dropout_degree_course` int(5) DEFAULT 0 COMMENT '중도탈락_학위과정(대학)',\
    `dropout_joint_operation` int(5) DEFAULT 0 COMMENT '중도탈락_교육과정공동운영생',\
    `dropout_course_language` int(5) DEFAULT 0 COMMENT '중도탈락_어학연수생',\
    `dropout_course_exchange` int(5) DEFAULT 0 COMMENT '중도탈락_교환학생',\
    `dropout_course_visiting` int(5) DEFAULT 0 COMMENT '중도탈락_방문학생',\
    `dropout_etc` int(5) DEFAULT 0 COMMENT '중도탈락_기타연수생',\
    `dropout_rate` float(5, 2) DEFAULT 0.00 COMMENT '외국학생 중도탈락율(중도탈락_총계/외국재적학생 총계)',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `foreign_dropout_status_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='외국학생 중도탈락 현황';"
table_dic['외국학생 중도탈락 현황'] = 'foreign_dropout_status'



# 2023년_ [4-사. 중도탈락 학생 현황_]
#################################################
# 테이블명 : dropout_status | 2023년_ [4-사. 중도탈락 학생 현황_]
# 재적학생 총계 | 사유별 중도탈락_총계 | 사유별 중도탈락_미등록 | 사유별 중도탈락_미복학 | 사유별 중도탈락_자퇴 | 사유별 중도탈락_학사경고 | 사유별 중도탈락_학생활동 | 사유별 중도탈락_유급제적 | 사유별 중도탈락_재학연한초과 | 사유별 중도탈락_기타 | 중도탈락 학생 비율(사유별 중도탈락_총계/재적학생 총계) | 신입생 총계 | 신입생_사유별 중도탈락_총계 | 
#################################################
table_dic['dropout_status'] = \
    "CREATE TABLE `dropout_status` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `enrolled_tot` int(5) DEFAULT 0 COMMENT '재적학생 총계',\
    `dropout_case_tot` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_총계',\
    `dropout_case_not_regist` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_미등록',\
    `dropout_case_not_return` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_미복학',\
    `dropout_case_self_dropout` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_자퇴',\
    `dropout_case_academic_warning` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_학사경고',\
    `dropout_case_student_activities` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_학생활동',\
    `dropout_case_withdrawal` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_유급제적',\
    `dropout_case_exceeded_age` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_재학연한초과',\
    `dropout_case_etc` int(5) DEFAULT 0 COMMENT '사유별 중도탈락_기타',\
    `dropout_case_rate` float(5, 2) DEFAULT 0.00 COMMENT '중도탈락 학생 비율(사유별 중도탈락_총계/재적학생 총계)',\
    `fresh_tot` int(5) DEFAULT 0 COMMENT '신입생 총계',\
    `fresh_dropout_case_tot` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_총계',\
    `fresh_dropout_case_not_regist` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_미등록',\
    `fresh_dropout_case_not_return` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_미복학',\
    `fresh_dropout_case_self_dropout` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_자퇴',\
    `fresh_dropout_case_academic_warning` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_학사경고',\
    `fresh_dropout_case_student_activities` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_학생활동',\
    `fresh_dropout_case_withdrawal` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_유급제적',\
    `fresh_dropout_case_exceeded_age` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_재학연한초과',\
    `fresh_dropout_case_etc` int(5) DEFAULT 0 COMMENT '신입생_사유별 중도탈락_기타',\
    `fresh_dropout_case_rate` float(5, 2) DEFAULT 0.00 COMMENT '신입생_중도탈락 학생 비율(신입생_사유별 중도탈락 학생_총계/재적학생 총계)',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `dropout_status_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='중도탈락 학생 현황';"
table_dic['중도탈락 학생 현황'] = 'dropout_status'




# 2023년_ [4-자. 신입생의 출신 고등학교 유형별 현황_]
#################################################
# 테이블명 : fresh_highschool_type | 2023년_ [4-자. 신입생의 출신 고등학교 유형별 현황_]
# 총입학자수 | 유형별_일반고_학생수 | 유형별_일반고_비율 | 유형별_특수목적_과학고_학생수 | 유형별_특수목적_과학고_비율 | 유형별_특수목적_외고국제고_학생수 | 유형별_특수목적_외고국제고_비율 | 유형별_특수목적_예체고_학생수 | 유형별_특수목적_예체고_비율 | 유형별_특수목적_산업수요맞춤형고_학생수 | 유형별_특수목적_산업수요맞춤형고_비율 | 유형별_특성화고_학생수 | 유형별_특성화고_비율 | 
#################################################
table_dic['fresh_highschool_type'] = \
    "CREATE TABLE `fresh_highschool_type` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `admission_tot` int(5) DEFAULT 0 COMMENT '총입학자수',\
    `type_general_sum` int(5) DEFAULT 0 COMMENT '유형별_일반고_학생수',\
    `type_general_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_일반고_비율',\
    `type_special_science_sum` int(5) DEFAULT 0 COMMENT '유형별_특수목적_과학고_학생수',\
    `type_special_science_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_특수목적_과학고_비율',\
    `type_special_foreign_sum` int(5) DEFAULT 0 COMMENT '유형별_특수목적_외고국제고_학생수',\
    `type_special_foreign_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_특수목적_외고국제고_비율',\
    `type_special_artsports_sum` int(5) DEFAULT 0 COMMENT '유형별_특수목적_예체고_학생수',\
    `type_special_artsports_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_특수목적_예체고_비율',\
    `type_special_industry_sum` int(5) DEFAULT 0 COMMENT '유형별_특수목적_산업수요맞춤형고_학생수',\
    `type_special_industry_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_특수목적_산업수요맞춤형고_비율',\
    `type_specialization_sum` int(5) DEFAULT 0 COMMENT '유형별_특성화고_학생수',\
    `type_specialization_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_특성화고_비율',\
    `type_autonomy_private_sum` int(5) DEFAULT 0 COMMENT '유형별_자율고_사립_학생수',\
    `type_autonomy_private_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_자율고_사립_비율',\
    `type_autonomy_public_sum` int(5) DEFAULT 0 COMMENT '유형별_자율고_공립_학생수',\
    `type_autonomy_public_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_자율고_공립_비율',\
    `type_etc_genius_sum` int(5) DEFAULT 0 COMMENT '유형별_기타_영재학교_학생수',\
    `type_etc_genius_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_기타_영재학교_비율',\
    `type_etc_foreigner_sum` int(5) DEFAULT 0 COMMENT '유형별_기타_외국인학교_학생수',\
    `type_etc_foreigner_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_기타_외국인학교_비율',\
    `type_etc_foreigncountry_sum` int(5) DEFAULT 0 COMMENT '유형별_기타_외국고등학교_학생수',\
    `type_etc_foreigncountry_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_기타_외국고등학교_비율',\
    `type_etc_other_sum` int(5) DEFAULT 0 COMMENT '유형별_기타_그외기타_학생수',\
    `type_etc_other_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_기타_그외기타_비율',\
    `type_sum` int(5) DEFAULT 0 COMMENT '유형별_소계_학생수',\
    `type_rate` float(5, 2) DEFAULT 0.00 COMMENT '유형별_소계_비율',\
    `location_specialcity_sum` int(5) DEFAULT 0 COMMENT '지역별_특별시_학생수',\
    `location_specialcity_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_특별시_비율',\
    `location_metropolitan_sum` int(5) DEFAULT 0 COMMENT '지역별_광역시,특별자치시_학생수',\
    `location_metropolitan_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_광역시,특별자치시_비율',\
    `location_medium_sum` int(5) DEFAULT 0 COMMENT '지역별_중소도시_학생수',\
    `location_medium_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_중소도시_비율',\
    `location_small_sum` int(5) DEFAULT 0 COMMENT '지역별_읍면_학생수',\
    `location_small_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_읍면_비율',\
    `location_special_sum` int(5) DEFAULT 0 COMMENT '지역별_특수_학생수',\
    `location_special_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_특수_비율',\
    `location_etc_sum` int(5) DEFAULT 0 COMMENT '지역별_기타_학생수',\
    `location_etc_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_기타_비율',\
    `location_sum` int(5) DEFAULT 0 COMMENT '지역별_소계_학생수',\
    `location_rate` float(5, 2) DEFAULT 0.00 COMMENT '지역별_소계_비율',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `fresh_highschool_type_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='신입생의 출신 고등학교 유형별 현황';"
table_dic['신입생의 출신 고등학교 유형별 현황'] = 'fresh_highschool_type'






# # XXXX
# #################################################
# # 테이블명 : XXXXX | XXXXX
# # 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL   | completed_stud_dept | completed_stud_human_social | completed_stud_natural_science | completed_stud_engineering | completed_stud_medicine | support_amount_stud_tot | support_amount_prototype | support_amount_education |
# # 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교명 | 이수학생수_해당학과 | 이수학생수_타학과_인문사회 | 이수학생수_타학과_자연과학 | 이수학생수_타학과_공학 | 이수학생수_타학과_의학 | 이수학생수_타학과_예체능 | 지원금수령_학생수(명) | 지원금수령_금액_시제품제작비 | 지원금수령_금액_교육프로그램지원비
# #################################################
# table_dic['XXXX'] = \
#     "CREATE TABLE `XXXXX` (\
#     `year` int(4) NOT NULL COMMENT '기준연도',\
#     `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
#     `` int(5) DEFAULT 0 COMMENT '',\
#     `` int(5) DEFAULT 0 COMMENT '',\
#     `` int(5) DEFAULT 0 COMMENT '',\
#     `` int(5) DEFAULT 0 COMMENT '',\
#     `` int(5) DEFAULT 0 COMMENT '',\
        

#     PRIMARY KEY (`year`,`univ_cd`),\
#     KEY `univ_cd` (`univ_cd`),\
#     CONSTRAINT `XXXXX_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
#     ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='XXXX';"
# table_dic['XXXXX'] = 'XXXXX'



# 2023년_ [4-다. 신입생 충원 현황_] 테이블
#################################################
# 테이블명 : new_students | 신입생 충원 현황
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL | admission_tot |  recruit_tot | recruit_in      | recruit_out     | applicant_tot | applicant_in  | applicant_out | admitted_tot | admitted_in_m | admitted_in_f   | admitted_out_m   | admitted_out_f   | recruit_rate       | competition_rate
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교 | 입학정원      | 모집인원(계) | 모집인원(정원내)  | 모집인원(정원외) | 지원자(계)     | 지원자(정원내) | 지원자(정원외) | 입학자(계) | 입학자(정원내)-남 | 입학자(정원내)-녀 | 입학자(정원외)-남 | 입학자(정원외)-녀 | 정원내 신입생 충원율 | 경쟁률
#################################################
table_dic['new_students'] = \
    "CREATE TABLE `new_students` (\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `admission_tot` int(5) DEFAULT 0 COMMENT '입학정원',\
    `recruit_tot` int(5) DEFAULT 0 COMMENT '모집인원(계)',\
    `recruit_in` int(5) DEFAULT 0 COMMENT '모집인원(정원내)',\
    `recruit_out` int(5) DEFAULT 0 COMMENT '모집인원(정원외)',\
    `applicant_tot` int(5) DEFAULT 0 COMMENT '지원자(계)',\
    `applicant_in` int(5) DEFAULT 0 COMMENT '지원자(정원내)',\
    `applicant_out` int(5) DEFAULT 0 COMMENT '지원자(정원외)',\
    `admitted_tot` int(5) DEFAULT 0 COMMENT '입학자(계)',\
    `admitted_in_m` int(5) DEFAULT 0 COMMENT '입학자(정원내)-남',\
    `admitted_in_f` int(5) DEFAULT 0 COMMENT '입학자(정원내)-녀',\
    `admitted_out_m` int(5) DEFAULT 0 COMMENT '입학자(정원외)-남',\
    `admitted_out_f` int(5) DEFAULT 0 COMMENT '입학자(정원외)-녀',\
    `recruit_rate` float(5, 2) DEFAULT 0.00 COMMENT '정원내 신입생 충원율[입학자(정원내)/모집인원(정원내)]',\
    `competition_rate` float(5, 2) DEFAULT 0.00 COMMENT '경쟁률[지원자(정원내)/모집인원(정원내)]',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`year`,`univ_cd`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `new_students_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='신입생 충원 현황';"
table_dic['신입생 충원 현황'] = 'new_students'








In [17]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys, os

#file = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료'
#sys.path.append('D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료')

#from tables_schema import *
#from tables_schema import tables_schema
#import tables_schema

#################################################
# 테이블의 존재 여부를 확인
#################################################
def chk_table(curs, dbname, table):
    _dbg_(table)
    sql = "select 1 from INFORMATION_SCHEMA.TABLES where TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}'".format(dbname, table)
    #print('chk_table : ', sql)
    ret = curs.execute(sql)
    
    if ret != 1:
        _dbg_('{}.{} 테이블이 없으므로 생성해야함'.format(dbname, table))
    else:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))

    return ret

#################################################
# 문서명에 해당하는 테이블 명을 확인, 띄어쓰기 모두 포함된 상태
#################################################
def get_table_name(doc_title):
    _dbg_('[{}]'.format(doc_title))
    _dbg_(table_dic.get(doc_title))
    
    if table_dic.get(doc_title) == None:
        print('[종료] [{}] 정보를 저장할 테이블 구조가 없으므로 [table_dic]에 구조부터 생성해야함'.format(doc_title))
        sys.exit(0)
    else:
        table_name = table_dic[doc_title]
        print('[{}] 정보를 저장할 [{}] 테이블 존재함'.format(doc_title, table_name))
    
    return table_name


#################################################
# 테이블 생성
#################################################
def create_table(curs, dbname, table):
    _dbg_(table)
    ret_val = ''
    ret_mode = '' #idx 가 있냐 없냐...

    sql = table_dic[table]
    chk_tbl = chk_table(curs, dbname, table)
    if chk_tbl != 1: #테이블명에 해당하는 테이블이 없음
        _dbg_('create_table : ', sql)
        rs = curs.execute(sql)
        
        if rs !=0 :
            _dbg_('[오류_종료]create_table 생성오류 [{}]: {}'.format(rs, table))
            os.exit(0)
        else:
            _dbg_('{}.{} 신규 테이블 생성 성공'.format(dbname, table))
            ret_val = 'create'
    elif chk_tbl == 1:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))
        ret_val = 'exist'
    else:
        _dbg_('[오류_종료] 알수없는 생성오류 [{}]: {}'.format(rs, table))
        
    #_dbg_(type(sql), sql)
    
    if sql.find('`idx` int auto_increment') > 0:
        _dbg_('인덱스가 있는 mode')
        ret_mode = 'idx_on'
    else:
        _dbg_('기준년도+학교코드 mode')
        ret_mode = 'idx_off'
    #    테이블에 idx가 있는 경우는 다르게 처리
    #    테이블에 idx가 있다면 인서트_업데이트를 다르게..
        
        
    return ret_val, ret_mode
    
    
#################################################
# 대학코드 -> 대학명
#################################################
def get_univ_name(curs, univ_code):
    _dbg_(univ_code)
    sql = 'SELECT {} FROM {} WHERE univ_cd = {}'.format('univ_name', 'university', univ_code) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_name = curs.fetchall() #university에 등록된 대학교 목록

    return univ_name
        
        
#################################################
# 대학명 -> 대학코드
#################################################
def get_univ_name(curs, univ_name):
    _dbg_(univ_name)
    sql = 'SELECT {} FROM {} WHERE univ_name_arlimi = {}'.format('univ_cd', 'university', univ_name) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_code = curs.fetchall() #university에 등록된 대학교 목록

    return univ_code
    
#################################################
# DB 테이블의 컬럼명과 데이터타입만 추출
#################################################
def get_db_column_name(curs, dbname, table):
    _dbg_(table)
    #cnt_columns = len(df_excel.columns) #엑셀 df읠 필드명

    sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table)

    curs.execute(sql_col)
    #conn.commit()
    col_info = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    
    _dbg_('col_info : ', type(col_info), col_info)

    idx_mode = 'off' #기준연도+학교코드 모드와 idx 모드로 나뉨
    col_name_list = list()
    col_name_str = ''
    col_cnt = len(col_info)
    
    for n in range(col_cnt):
        col = col_info[n]['COLUMN_NAME']
        #_dbg_('[{}]'.format(col))
        if col == 'idx':
            idx_mode = 'on'
        else:
            #col_name_list.append(col_name[n]['COLUMN_NAME'])
            col_name_list.append(col)
            #_dbg_('appended : [{}]'.format(col))

    if idx_mode == 'on': #idx하나를 빼야 함
        col_cnt = col_cnt - 1

    for n in range(col_cnt):
        if n == 0:
            col_name_str = '`' + str(col_name_list[n]) + '`, '
        elif n >= col_cnt-1:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`'
        else:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`, '


    #print(col_name_str)
    # print(type(col_name), len(col_name), col_name)
    # print(type(col_name[0]), len(col_name[0]), col_name[0])
    # print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])

    return col_info, col_name_str, col_name_list


#################################################
# 데이터 프레임의 의 컬럼명만 추출
#################################################
# def get_df_column_name(curs, dbname, table):





#################################################
# df를 실제 테이블에 입력
#################################################
def insert_df(curs, conn, df_excel, dbname, table):
    _dbg_(table)
    # data insert to db

    field_names = ''
    field_data_str = ''

    #테이블의 헤더정보만 추출    
    col_info, col_name_str, col_name_list = get_db_column_name(curs, dbname, table)

    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 인서트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        #print(field_data)
        #print(field_data[0])
        
        #입력할 레코드 생성
        for m in range(len(col_name_list)):
            if m == 0: #2023, 기준연도
                field_data_str = str(field_data[m])            
            elif m == 1: # "0000069", 학교코드
                field_data_str = field_data_str + ', "' + str(field_data[m] + '"')
            elif m == 2: # "1학기", 학기 ----> 전공과목 성적분포만 해당됨  ->> 굳이 필요한가?
                field_data_str = field_data_str + ', "' + str(field_data[m] + '"')                
            else:
                field_data_str = field_data_str + ', ' + str(field_data[m]) 
        
        print(type(field_data_str), field_data_str)
        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, field_data_str)
        print(n, field_data[1], sql_insert)
        curs.execute(sql_insert)
    conn.commit()

    # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    # print(sql_insert)


    #print(sql)
    # for idx in range(len(df)):
    #     	curs.execute(sql_insert, tuple(df.values[idx]))
    # conn.commit()


#################################################
# 테이블 필드명의 데이터 타입을 얻어옴
#################################################
def get_data_type(col_info, col_name):
    data_type = ''
    for info in col_info:
        #print(info['COLUMN_NAME'], info['DATA_TYPE'])
        if info['COLUMN_NAME'] == col_name:
            data_type = info['DATA_TYPE']
        
    return data_type


# {COLUMN_NAME': 'idx', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'year', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'univ_cd', 'DATA_TYPE': 'varchar'}


#################################################
# df를 기존 테이블에 업데이트, 신규나 변경된 데이터만 찾아서 업데이트하면 될듯...일단은 키값만 일치하면 모두 업데이트
#################################################
def insert_update_df(curs, conn, df_excel, dbname, table):
    _dbg_(table)
    cnt_insert = 0
    cnt_update = 0

    #DB 테이블의 헤더정보만 추출    
    col_info, col_name_str, col_name_list = get_db_column_name(curs, dbname, table)
    _dbg_(col_name_list)
    


    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        #print(field_data)
        #print(field_data[0])
        
        set_data = ''
        year = 0 
        univ_cd = ''
        semester = ''
        for m in range(len(col_name_list)):
            
            if col_name_list[m] == 'univ_cd' or col_name_list[m] == 'semester':
                set_data = set_data + col_name_list[m] + '= "' + str(field_data[m]) + '" , '
            elif m < len(col_name_list)-1:
                #print(m, len(col_name_list), col_name_list[m], len(field_data), str(field_data[m]))
                set_data = set_data + col_name_list[m] + '=' + str(field_data[m]) + ', '
            else:
                set_data = set_data + col_name_list[m] + '=' + str(field_data[m])
                
            if col_name_list[m] == 'year':
                year = field_data[m]
            elif col_name_list[m] == 'univ_cd':
                univ_cd = str(field_data[m])
            elif col_name_list[m] == 'semester':
                semester = str(field_data[m])
                
            #신규 인서트가 필요한 경우 레코드 데이터
            if m == 0: #2023, 기준연도
                field_data_str = str(field_data[m])            
            elif m == 1: # "0000069", 학교코드
                field_data_str = field_data_str + ', "' + str(field_data[m]) + '"'
            elif m == 2: # "1학기", 학기 ----> 전공과목 성적분포만 해당됨  ->> 굳이 필요한가?
                field_data_str = field_data_str + ', "' + str(field_data[m]) + '"'                
            else:
                field_data_str = field_data_str + ', ' + str(field_data[m])                 
              
        #_dbg_('테이블 = ' + table, 'year = ' + year, 'univ_cd = ' + univ_cd, 'semester = ' + semester)
        if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
        #if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
            #'select * from cosshubtest.major_grades where year = 2022 and univ_cd = "1000007" and semester = "2학기"'.format(dbname, table)
            sql = 'select * from {}.{} where year = {} and univ_cd = "{}" and semester = "{}"'.format(dbname, table, year, univ_cd, semester) #table_dic['전공과목 성적 분포'] = 'major_grades' 만 해당
        else:
            sql = 'select * from {}.{} where year = {} and univ_cd = "{}"'.format(dbname, table, year, univ_cd) #
        cnt = curs.execute(sql) #검색된 갯수
        result = curs.fetchall() #리스트 안에 딕셔너리로...
        #print(type(cnt), cnt, type(result), result)

        if cnt == 0: #쿼리 결과 없음 -> 신규 레코드라는 의미
            #print(cnt, '쿼리 검색 결과 없음(신규 인서트 필요) : ', sql)
            sql = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, field_data_str)
            cnt_insert += 1
        elif cnt == 1: #정상 검색 -> 업데이트 대상
            #print(cnt, '쿼리 검색 성공(업데이트) ')
            if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
                sql = 'update {}.{} set {} where year = {} and univ_cd = "{}" and semester = "{}"'.format(dbname, table, set_data, year, univ_cd, semester)
            else:
                sql = 'update {}.{} set {} where year = {} and univ_cd = "{}"'.format(dbname, table, set_data, year, univ_cd)
            cnt_update += 1
        elif cnt >= 2: #키값으로 검색했는데 결과가 2개 이상이라면 문제가 있음
            print(cnt, '오류 : 검색 결과가 너무 많음(키 오류) : ', sql)
            sys.exit(0)
        # 각 셀들을 비교해서 틀린 셀 값만 찾아서 변경된 부분만 업데이트해야하지만 나중에.....        
        # elif cnt == 1: #키값으로 1개만 검색되었으므로 업데이트 대상
        #     key_list = list(result.keys())
        #     for k in key_list:
        #         if result[k] == 
        
        #print(type(field_data_str), field_data_str)
        #
        
        #print(n, field_data[1], sql)
        curs.execute(sql)
    conn.commit()
    
    print(table, '테이블 : ', cnt_insert, '개의 신규 인서트     ', cnt_update, '개의 업데이트')




#################################################
# 기준년도 + 학교코드가 키 값이 아니라 idx기준을 데이터 인서트
#################################################
def insert_update_df_idx(curs, conn, df_excel, dbname, table, mode = 'create'):
    _dbg_(table)
    cnt_insert = 0
    cnt_update = 0

    #DB 테이블의 헤더정보만 추출    
    col_info, col_name_org, col_name_list = get_db_column_name(curs, dbname, table) #리턴값중 col_info만 사용

    # 테스트코드 데이터 타입만 읽어오기    
    # data_type = get_data_type(col_info, 'year')
    # _dbg_(data_type)
    
    _dbg_(len(col_info), col_info)
    _dbg_(len(col_name_list), col_name_list)
    # _dbg_(col_name_str)
    
    
    # reg_date와 mod_date는 처리에서 제외
    col_info_cnt = len(col_info)
    for i in range(len(col_info)):
        if col_info[i]['COLUMN_NAME'] == 'reg_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1
        elif col_info[i]['COLUMN_NAME'] == 'mod_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1

    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        #print(field_data)
        #print(field_data[0])
        
        # set_data = ''
        # year = 0 
        # univ_cd = ''
        # semester = ''
        
        first_field = True #sql생성을 위해 처음과 중간 끝 작업을 위해
        idx_cnt = 0 #idx는 자동증가이기 때문에 처리하지 않기때문에 field_data[m]과 -1개 차이가 발생함
        col_name_str = ''
        col_data_str = ''
        select_where = ''
        update_data = ''
        
        
        for m in range(col_info_cnt):
            
            #_dbg_(m, col_info_cnt, len(col_info))
            
            #idx는 자동증가이므로 제외
            col_name = col_info[m]['COLUMN_NAME']
            col_type = col_info[m]['DATA_TYPE']
            
            #_dbg_(col_name, col_type, field_data[m-idx_cnt])
            
            #_dbg_(col_name, col_type)
            if(col_name == 'idx'):
                _dbg_("idx mode ON")
                idx_cnt = 1
            else: #idx이외의 나머지 필드
                if first_field == True:
                    #_dbg_("처음")  #idx가 있으면 하나 빼야함!!
                    col_name_str = '`{}`, '.format(col_name) #처음
                    if col_type == 'int':
                        col_data_str = '{}, '.format(field_data[m-idx_cnt])
                        select_where = '`{}` = {} and '.format(col_name, field_data[m-idx_cnt])
                        #update_data = '{} = {}, '.format(col_name_str, field_data[m])
                        
                    elif col_type == 'float':
                        col_data_str = '{}, '.format(field_data[m-idx_cnt])
                        select_where = '`{}` = {} and '.format(col_name, field_data[m-idx_cnt])
                        
                    elif col_type == 'varchar':
                        col_data_str = '"{}", '.format(field_data[m-idx_cnt])
                        select_where = '`{}` = "{}" and '.format(col_name, field_data[m-idx_cnt])
                    first_field = False

                elif m >= col_info_cnt-1: #마지막
                    #_dbg_("마지막", m)
                    col_name_str = col_name_str + '`{}` '.format(col_name) #마지막
                    if col_type == 'int':
                        col_data_str = col_data_str + '{}'.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = {}'.format(col_name, field_data[m-idx_cnt])
                        
                    elif col_type == 'float':
                        col_data_str = col_data_str + '{}'.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = {}'.format(col_name, field_data[m-idx_cnt])
                        
                    elif col_type == 'varchar':
                        col_data_str = col_data_str + '"{}"'.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = "{}"'.format(col_name, field_data[m-idx_cnt])

                else:
                    #_dbg_("중간", m)
                    col_name_str = col_name_str + '`{}`, '.format(col_name) #중간
                    if col_type == 'int':
                        col_data_str = col_data_str + '{}, '.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = {} and '.format(col_name, field_data[m-idx_cnt])
                        
                    elif col_type == 'float':
                        col_data_str = col_data_str + '{}, '.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = {} and '.format(col_name, field_data[m-idx_cnt])
                        
                    elif col_type == 'varchar':
                        col_data_str = col_data_str + '"{}", '.format(field_data[m-idx_cnt])
                        select_where = select_where + '`{}` = "{}" and '.format(col_name, field_data[m-idx_cnt])


        #기존 테이블이 존재하면 업데이트 모드로 해야함            
        #테이블은 존재하지만 실제 데이터는 없을수도 있음
        if mode == 'exist':
            sql = 'select * from {}.{} where {}'.format(dbname, table, select_where) #table_dic['전공과목 성적 분포'] = 'major_grades' 만 해당
            #_dbg_(select_where)
            #_dbg_(sql)
            #_dbg_(col_data_str)
            #_dbg_(field_data)
            
            cnt = curs.execute(sql) #검색된 갯수
            result = curs.fetchall() #리스트 안에 딕셔너리로...
            #print(type(cnt), cnt, type(result), result)

            ### Ryu, 업데이트시 키값만 정확하게 읽어오는 거 필요
            if cnt == 1: #정상 검색 -> 업데이트 대상    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 키값 기준으로 검색을 해서 찾아야하는데 idx 모드일때 등 키값을 정확히 찾기가 어려움... 천천히 생각
                #print(cnt, '쿼리 검색 성공(업데이트) ')
                #if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
                #sql = 'update {}.{} set {} where {}'.format(dbname, table, col_name_str, select_where)
                #_dbg_(cnt, '업데이트를 하면 좋은데 키값 기준에 대한 고민이 필요')
                #_dbg_(cnt, '기존 데이터 있음(업데이트 실행) : ', sql)
                # else:
                #     sql = 'update {}.{} set {} where year = {} and univ_cd = "{}"'.format(dbname, table, set_data, year, univ_cd)
                cnt_update += 1
            elif cnt == 0: #테이블은 존재하지만 데이터는 없는 경우이기 때문에 인서트를 해야함
                sql = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, col_data_str)
                _dbg_('exist : ', sql)
                cnt_insert += 1
            else:
                _dbg_(cnt, '오류 : 테이블이 존재하지만 알수 없는 오류 : ', sql)
                sys.exit(0)

        elif mode == 'create': 
            sql = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, col_data_str)
            _dbg_('create : ', sql)
            cnt_insert += 1
        else:
            _dbg_(cnt, '오류 : 정의되지 않음 ', sql)
            sys.exit(0)
        
        if sql != '':
            curs.execute(sql)
    conn.commit()
    
    print(table, '테이블 : ', cnt_insert, '개의 신규 인서트     ', cnt_update, '개의 업데이트')

In [19]:
##################################################################################################
# university 테이블, 단독작업
##################################################################################################

def create_university(dbname, curs, conn, reset_mode = False):
    excel_univ = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/05.설계/01.테이블설계/01.Codes.xlsx'
    sheet_name = '학교코드정보'

    table_name = 'university'

    df_univ = pd.read_excel(excel_univ, sheet_name = sheet_name)

    df_univ = df_univ.fillna('None')

    cols_ko = list(df_univ.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df_univ.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df_univ.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df_univ.drop(0) #특정행 삭제


    # 기존 테이블을 삭제 후 인서트하는 경우
    if reset_mode == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table_name)
        curs.execute(sql_truncate)
        conn.commit()

    ret, mode = create_table(curs, dbname, table = table_name)
    _dbg_('테이블 생성 정보 : ', ret, mode)

    if ret == 'create' or ret == 'exist': #create, exist
        #idx 모드와 무관하게 실행하자
        #insert_update_df_idx(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...
            
        # data insert to db
        field_names = ''
        field_data = ''

        for n in range(len(cols_en)):
            if n == 0 :
                field_names = cols_en[n]
                field_data = '%s'
            else:
                field_names = field_names + ', ' + cols_en[n]
                field_data = field_data + ', ' + '%s'
            
            
        # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
        # print(sql_insert)

        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_name, field_names, field_data)
        #print(sql)
        for idx in range(len(df)):
            curs.execute(sql_insert, tuple(df.values[idx]))
        conn.commit()

        print("\n#########################################################################################################")
        print('##### university 테이블 생성 완료 : 테이블 = {}, 엑셀파일={}'.format(table_name,  excel_univ))
        print("#########################################################################################################\n")

In [20]:
#################################################################################################################################################
#
# 인덱스 생성 후 숫자로 키값을 정하는 케이스
# 학교명 등 셀이 통합되어있음.....
#################################################################################################################################################

# 2023년_ [12-카-4-(1). 계약학과 설치 운영 현황_]
#################################################
# 테이블명 : contract_department | 계약학과 설치 운영 현황
# 필드 : year    | DEL      | DEL     | DEL  | DEL | DEL  | degree  | contract_department_type | department | parent_department | parent_department_tot | contract_department_name | courses_classification_type | contract_department_tot | fresh_stud_tot | enrolled_stud_tot | area    | use_external_learning  | contract_type 
#                                                        | expense_industry_rate    | expense_agency_rate        | expense_univ_rate      | expense_stud_rate     | expense_support_agency | expense_support_title | recruitment_graduate | recruitment_employed_contract  | recruitment_employed_other_industry | industry_tot
# 설명 : 기준연도 | 학교종류 | 설립구분 | 지역 | 상태 | 학교 | 취득학위 | 계약학과 유형             | 계열        | 모체학과 명칭 | 모체학과 입학정원           | 계약학과 명칭             | 신/편입과정 구분             | 계약학과 정원            | 입학생 수       | 재학생수           | 설치권역 | 외부학습장 이용여부     | 계약형태  
#                                                        | 경비부담_산업체경비_부담률 | 경비부담_외부기관경비_지원율 | 경비부담_대학자체_감면율 | 경비부담_학생경비_부담률 | 경비지원_기관명         | 경비지원_사업명        | 채용조건형_졸업자      | 채용조건형_취업자_계약산업체     | 채용조건형_취업자_타산업체    | 산업체수
#################################################

table_dic['contract_department'] = \
    "CREATE TABLE `contract_department` (\
    `idx` int auto_increment NOT NULL COMMENT '고유키 인덱스',\
    `year` int(4) NOT NULL COMMENT '기준연도',\
    `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',\
    `degree` varchar(50) COMMENT '취득학위',\
    `contract_department_type` varchar(50) COMMENT '계약학과 유형',\
    `department` varchar(50) COMMENT '계열',\
    `parent_department` varchar(50) COMMENT '모체학과 명칭',\
    `parent_department_tot` int(5) DEFAULT 0 COMMENT '모체학과 입학정원',\
    `contract_department_name` varchar(50) COMMENT '계약학과 명칭',\
    `courses_classification_type` varchar(50) COMMENT '신/편입과정 구분',\
    `contract_department_tot` int(5) DEFAULT 0 COMMENT '계약학과 정원',\
    `fresh_stud_tot` int(5) DEFAULT 0 COMMENT '입학생 수',\
    `enrolled_stud_tot` int(5) DEFAULT 0 COMMENT '재학생수',\
    `area` varchar(50) COMMENT '설치권역',\
    `use_external_learning` varchar(50) COMMENT '외부학습장 이용여부',\
    `contract_type` varchar(50) COMMENT '계약형태',\
    `expense_industry_rate` float(5, 2) DEFAULT 0.00 COMMENT '경비부담_산업체경비_부담률',\
    `expense_agency_rate` float(5, 2) DEFAULT 0.00 COMMENT '경비부담_외부기관경비_지원율',\
    `expense_univ_rate` float(5, 2) DEFAULT 0.00 COMMENT '경비부담_대학자체_감면율',\
    `expense_stud_rate` float(5, 2) DEFAULT 0.00 COMMENT '경비부담_학생경비_부담률',\
    `expense_support_agency` varchar(200) COMMENT '경비지원_기관명',\
    `expense_support_title` varchar(200) COMMENT '경비지원_사업명',\
    `recruitment_graduate` varchar(50) COMMENT '채용조건형_졸업자',\
    `recruitment_employed_contract` varchar(200) COMMENT '채용조건형_취업자_계약산업체',\
    `recruitment_employed_other_industry` varchar(200) COMMENT '채용조건형_취업자_타산업체',\
    `industry_tot` int(5) DEFAULT 0 COMMENT '산업체수',\
    `reg_date` datetime DEFAULT CURRENT_TIMESTAMP COMMENT '생성일시',\
    `mod_date` datetime DEFAULT NULL COMMENT '수정일시',\
    PRIMARY KEY (`idx`),\
    KEY `univ_cd` (`univ_cd`),\
    CONSTRAINT `contract_department_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)\
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='계약학과 설치 운영 현황';"
table_dic['계약학과 설치 운영 현황'] = 'contract_department'

In [21]:
##################################################################################################
# DB 및 작업 폴더 설정
##################################################################################################

# %load_ext autoreload
# %autoreload 2

dbname = 'swu_external'

#conn, curs = db_setting('133.186.247.101', 'root', 'coss!@#$', dbname = 'cosshubtest') #coss test
conn, curs = db_setting('13.209.213.214', 'root', 's!W@u#12', dbname = dbname)

sheet_name = 'Sheet1'
table = 'university'

template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/TEMPLATE'
excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/EXCEL'

template_file_list = read_filename(template_dir)
excel_file_list = read_filename(excel_dir)

# university 테이블 생성
# create_university(dbname, curs, conn, reset_mode = False)

In [24]:
excel_worked = [
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
]


for excel in excel_file_list:
    for template in template_file_list:
        if excel[4:] == template[4:]:
            print("\n\n")
            print("#########################################################################################################")
            print("##### 작업 엑셀 파일명 : ", excel)
            print("#########################################################################################################")

            template_file = '{}/{}'.format(template_dir, template)
            excel_file = '{}/{}'.format(excel_dir, excel)
            
            df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용
            
            df_excel = remove_empty_row1(df_excel, df_template_shape0)
            #df_excel = refine_header(df_excel)
            df_excel, find_year = remove_empty_row2(df_excel)

            df_excel = remove_univ_col(df_excel)
            df_excel = fill_year_col(df_excel, find_year, col_name='기준연도')
            df_excel = fill_univ_name(df_excel, col_name='학교명') # 전공과목 성적 분포_학교별자료.xlsx 특화
            
            df_excel = strip_univ_name(df_excel, col_name='학교명')
            df_excel = set_univ_code(curs, conn, df_excel, dbname, excel_file, sheet_name) #데이터프레임의 학교명을 코드명으로 변경, DB학교 공백처리...
            
            # 이제 데이터를 DB에 입력하면 됨
            _dbg_('{}에 데이터 입력 시작 : {}'.format(table_dic.get(doc_title), doc_title))
            table_name = get_table_name(doc_title)
            _dbg_('{}에 데이터 입력 시작 : {}'.format(table_dic.get(doc_title), doc_title))
            
            ret, mode = create_table(curs, dbname, table = table_name)
            if ret == 'create' or ret == 'exist': #create, exist
                # if mode == 'idx_off':
                #     insert_update_df(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...
                # elif mode == 'idx_on':
                #     print('idx_on 모드로 해야함')
                #     insert_update_df_idx(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...

                #idx 모드와 무관하게 실행하자
                insert_update_df_idx(curs, conn, df_excel, dbname, table = table_name, mode = ret) #df_excel을 table에 잘 비교해서 업데이트...

            print("\n#########################################################################################################")
            print('##### 작업 완료 : 테이블 = {}, 문서명={}, 엑셀파일={}'.format(table_name, doc_title, excel_file))
            print("#########################################################################################################\n")

            dict_item = {'doc_title' : '{}'.format(doc_title), 'db_table' : '{}'.format(table_name), 'excel_file' : '{}'.format(excel_file)}
            excel_worked.append(dict_item)


if len(excel_worked) <= 0:
    print("\n#########################################################################################################")
    print('##### 작업대상 파일이 없음 : {}개'.format(len(excel_worked)))
    print("#########################################################################################################\n")    
else:
    print("\n#######################################################################################################################################################################")
    print('##### 전체 작업 완료 : {}개'.format(len(excel_worked)))
    print("#######################################################################################################################################################################\n")

    for n in range(len(excel_worked)):
        print('\n----- ', n+1, '번째 작업파일 정보 ------')
        print('문서명 : ', excel_worked[n]['doc_title'])
        print('테이블 : ', excel_worked[n]['db_table'])
        print('파일명 : ', excel_worked[n]['excel_file'])
            
            





#########################################################################################################
##### 작업 엑셀 파일명 :  2023년 _대학_12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_학교별자료.xlsx
#########################################################################################################
[dbg] compare_schema  :  D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/TEMPLATE/0000년 _대학_12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_학교별자료.xlsx D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/EXCEL/2023년 _대학_12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_학교별자료.xlsx Sheet1  
[dbg] compare_schema  :  doc_title :  14 37 2023년_ [12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_] [캡스톤 디자인(창의적 설계) 운영 현황]
[dbg] compare_schema  :  작업 문서명  2023년_ [12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_] [캡스톤 디자인(창의적 설계) 운영 현황]  
[dbg] remove_empty_row1  :  df_template_shape0 :  5   
[dbg] remove_empty_row1  :  header_info :  <class 'list'> ['2023년_ [12-카-2. 캡스톤 디자인(창의적 설계) 운영 현황_]', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 

ProgrammingError: (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'and `univ_cd` = "0000072" and `semester` = "1학기" and `stud_tot` = 27974 a...\' at line 1')

In [ ]:
sys.exit(0)

In [ ]:
template_file = '{}/{}'.format(template_dir, template)
excel_file = '{}/{}'.format(excel_dir, excel)

df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용

df_excel = remove_empty_row1(df_excel, df_template_shape0)
df_excel = refine_header(df_excel)
df_excel, find_year = remove_empty_row2(df_excel)

In [ ]:
#!pip install pandas
%pip install gensim

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Load pre-trained Word2Vec model (you need to download the model first)
model_path = "path_to_your_word2vec_model.bin"
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)

def word_similarity(word1, word2):
    try:
        similarity = word_vectors.similarity(word1, word2)
        return similarity
    except KeyError as e:
        return f"Word '{e.args[0]}' not in vocabulary."

word1 = "apple"
word2 = "banana"
similarity_score = word_similarity(word1, word2)
print(f"Similarity between '{word1}' and '{word2}': {similarity_score}")


In [ ]:
df_excel.head(6)

In [ ]:
df_excel.tail(6)

In [ ]:
# data insert to db

field_names = ''
field_data = ''

for n in range(len(cols_en)):
    if n == 0 :
        field_names = cols_en[n]
        field_data = '%s'
    else:
        field_names = field_names + ', ' + cols_en[n]
        field_data = field_data + ', ' + '%s'
        
        
# sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
# print(sql_insert)

sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table, field_names, field_data)
#print(sql)
for idx in range(len(df)):
    	curs.execute(sql_insert, tuple(df.values[idx]))
conn.commit()


# 중도탈락 현황을 DB에 등록
4-사. 중도탈락 학생 현황.xlsx + 4-바-2. 외국학생 중도탈락 현황.xlsx 을 통합

-- cosshubtest.dropout_stat definition

CREATE TABLE `dropout_stat` (
  `year` int(4) NOT NULL COMMENT '기준연도',
  `univ_cd` varchar(10) NOT NULL COMMENT '대학 고유코드',
  `stud_tot` int(5) DEFAULT 0 COMMENT '재적_학생수',
  `stud_sum` int(5) DEFAULT 0 COMMENT '재적_중도탈락_합계',
  `stud_unregist` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미등록',
  `stud_unreturn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_미복학',
  `stud_drop` int(5) DEFAULT 0 COMMENT '재적_중도탈락_자퇴',
  `stud_warn` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학사경고',
  `stud_act` int(5) DEFAULT 0 COMMENT '재적_중도탈락_학생활동',
  `stud_expulsion` int(5) DEFAULT 0 COMMENT '재적_중도탈락_유급제적',
  `stud_attendyear` int(5) DEFAULT 0 COMMENT '재적_중도탈락_재학연한초과',
  `stud_etc` int(5) DEFAULT 0 COMMENT '재적_중도탈락_기타',
  `stud_rate` float(5,2) DEFAULT 0.00 COMMENT '재적_중도탈락_비율',
  `fresh_tot` int(5) DEFAULT 0 COMMENT '신입_학생수',
  `fresh_sum` int(5) DEFAULT 0 COMMENT '신입_중도탈락_합계',
  `fresh_unregist` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미등록',
  `fresh_unreturn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_미복학',
  `fresh_drop` int(5) DEFAULT 0 COMMENT '신입_중도탈락_자퇴',
  `fresh_warn` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학사경고',
  `fresh_act` int(5) DEFAULT 0 COMMENT '신입_중도탈락_학생활동',
  `fresh_expulsion` int(5) DEFAULT 0 COMMENT '신입_중도탈락_유급제적',
  `fresh_attendyear` int(5) DEFAULT 0 COMMENT '신입_중도탈락_재학연한초과',
  `fresh_etc` int(5) DEFAULT 0 COMMENT '신입_중도탈락_기타',
  `fresh_rate` float(5,2) DEFAULT 0.00 COMMENT '신입_중도탈락_비율',
  `foreign_tot` int(5) DEFAULT 0 COMMENT '외국인_학생수',
  `foreign_sum` int(5) DEFAULT 0 COMMENT '외국인_중도탈락_합계',
  `foreign_degree` int(5) DEFAULT 0 COMMENT '외국인_학위과정(대학)',
  `foreign_common` int(5) DEFAULT 0 COMMENT '외국인_교육과정공동운영생',
  `foreign_lang` int(5) DEFAULT 0 COMMENT '외국인_어학연수생',
  `foreign_exchange` int(5) DEFAULT 0 COMMENT '외국인_교환학생',
  `foreign_visit` int(5) DEFAULT 0 COMMENT '외국인_방문학생',
  `foreign_etc` int(5) DEFAULT 0 COMMENT '외국인_기타연수생',
  `foreign_rate` float(5,2) DEFAULT 0.00 COMMENT '외국인_중도탈락_비율',
  PRIMARY KEY (`year`,`univ_cd`),
  KEY `univ_cd` (`univ_cd`),
  CONSTRAINT `dropout_ibfk_1` FOREIGN KEY (`univ_cd`) REFERENCES `university` (`univ_cd`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COMMENT='중도탈락 학생 현황';

## 입력되는 엑셀의 기본 조건

1. 제목 행은 총 4행
- 1행 : 2023년_ [4-마. 재적 학생 현황_]
- 2행 ~ 4행 : 제목행

2. 공통열은 university 테이블 참조
- '설립구분'
- '학교종류'
- '지역'
- '상태'

## 중도탈락 엑셀을 DB로 인서트
- 한국 학생 + 외국학생 두개의 파일
- 기준연도	학교종류	설립구분	지역	상태	학교	재적학생\n(A)	사유별 중도탈락 학생 계(B)	미등록	미복학	...	사유별 중도탈락 학생(신입생) 계(B`)	미등록	미복학	자퇴	학사경고	학생활동	유급제적	재학연한초과	기타	중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
- 기준연도	학교종류	설립구분	지역	상태	학교	외국 재적학생 총계(A)	학위과정(대학)	교육과정공동운영생	어학연수생	...	방문학생	기타연수생	중도탈락 학생 총계(B)	학위과정(대학)	교육과정공동운영생	어학연수생	교환학생	방문학생	기타연수생	외국 학생 중도탈락률(%)(B/A)×100



In [ ]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd


# DB 접속 정보
ip = '133.186.247.101'
id = 'root'
pwd = 'coss!@#$'

dbname = 'cosshubtest'  ## 변경 필요
table_univ = 'university'    ## 변경 필요
table_dropout = 'dropout_stat'    ## 변경 필요


excel_file1 = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-사. 중도탈락 학생 현황 - 2022.xlsx'
sheet_name1 = '원본'

excel_file2 = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/1.학생/4-바-2. 외국학생 중도탈락 현황 - 2022.xlsx'
sheet_name2 = '원본'


#%pip install pymysql
import pymysql

conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
curs = conn.cursor(pymysql.cursors.DictCursor)

if conn == None:
    print('DB 연결 오류')
elif curs == None:
    print('curs 연결 오류')


# excel file load
#%pip install openpyxl
df1 = pd.read_excel(excel_file1, sheet_name = sheet_name1)
df2 = pd.read_excel(excel_file2, sheet_name = sheet_name2)


print(excel_file1)
print(excel_file2)

In [ ]:
##########################################################################
# 중도탈락 학생 현황 필드 변경 (0번째와 1번째 데이터 행을 타이틀행으로 변경)
# 엑셀의 제목행은 기본 두줄이므로 두줄을 통합해야함

# 2023년_ [4-사. 중도탈락 학생 현황]
# 제목0
# 제목1
##########################################################################

#df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
#df.columns('a', 'b') 

cols_field = list(df1.columns) #컬럼 시리즈를 리스트로 변경
cols_data0 = list(df1.iloc[0]) #데이터0 시리즈를 리스트로 변경
cols_data1 = list(df1.iloc[1]) #데이터1 시리즈를 리스트로 변경

for n in range(len(cols_field)):
    if pd.isna(cols_data0[n]) == True and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data1[n]
        
    elif pd.isna(cols_data0[n]) == False and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data0[n] + ' ' + cols_data1[n]        
    
for n in range(len(cols_field)):
    df1.rename(columns={cols_field[n]:cols_data0[n]}, inplace=True) #특정 컬럼명 변경
    
df1 = df1.drop(0) #제목 3행중 2행 삭제
df1 = df1.drop(1) #제목 3행중 2행 삭제

##########################################################################
cols_field = list(df2.columns) #컬럼 시리즈를 리스트로 변경
cols_data0 = list(df2.iloc[0]) #데이터0 시리즈를 리스트로 변경
cols_data1 = list(df2.iloc[1]) #데이터1 시리즈를 리스트로 변경

for n in range(len(cols_field)):
    if pd.isna(cols_data0[n]) == True and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data1[n]
        
    elif pd.isna(cols_data0[n]) == False and pd.isna(cols_data1[n]) == False:
        cols_data0[n] = cols_data0[n] + ' ' + cols_data1[n]        
    
for n in range(len(cols_field)):
    df2.rename(columns={cols_field[n]:cols_data0[n]}, inplace=True) #특정 컬럼명 변경
    
df2 = df2.drop(0) #제목 3행중 2행 삭제
df2 = df2.drop(1) #제목 3행중 2행 삭제


In [ ]:
# 데이터프레임의 nan 값을 None으로 변경
#df.dropna(inplace=True)
#df = df.where((pd.notnull(df)), 'None')

df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [ ]:
df1

In [ ]:
df2

In [ ]:
df2.columns[12]

In [ ]:
# df2의 열 삭제를 위해 열 이름 변경

df2_del = df2.columns.tolist()

print(type(df2_del), len(df2_del), df2_del)

df2_del[7] = 'DEL'
df2_del[8] = 'DEL'
df2_del[9] = 'DEL'
df2_del[10] = 'DEL'
df2_del[11] = 'DEL'
df2_del[12] = 'DEL'


# 전체 열 이름 입력하기
df2.columns = df2_del

# 선택하여 열 이름 변경하기
#df.rename(columns={'Before':'After'})

print(type(df2_del), len(df2_del), df2_del)
print(df2.columns)


In [ ]:
# 중복 또는 불필요한 df의 열 삭제

df1 = df1.drop(['설립구분'], axis = 1)
df1 = df1.drop(['학교종류'], axis = 1)
df1 = df1.drop(['지역'], axis = 1)
df1 = df1.drop(['상태'], axis = 1)


#df2 = df2.drop(df2.columns[7], axis=1) # 학위과정(대학) 인데 동일한 컬럼명이 모두 삭제됨


df2 = df2.drop(['설립구분'], axis = 1)
df2 = df2.drop(['학교종류'], axis = 1)
df2 = df2.drop(['지역'], axis = 1)
df2 = df2.drop(['상태'], axis = 1)

df2 = df2.drop(['DEL'], axis = 1)



# df2 = df2.drop(['학위과정(대학)'], axis = 1)
# df2 = df2.drop(['교육과정공동운영생'], axis = 1)
# df2 = df2.drop(['어학연수생'], axis = 1)
# df2 = df2.drop(['교환학생'], axis = 1)
# df2 = df2.drop(['방문학생'], axis = 1)
# df2 = df2.drop(['기타연수생'], axis = 1)


In [ ]:
df1

In [ ]:
df2

## 두개의 파일을 두개의 키 값을 기준으로 결합

### 1. df1의 컬럼에 df2에서 가져올 컬럼으로 추가함
### 2. 연도+학교명 키값으로 두개의 df를 결합

In [ ]:
# 1. df1의 컬럼에 df2에서 가져올 컬럼으로 추가함

cols_field1 = list(df1.columns) #컬럼 시리즈를 리스트로 변경
cols_field2 = list(df2.columns) #컬럼 시리즈를 리스트로 변경

In [ ]:
print(cols_field1)
print(cols_field2)


In [ ]:
# 외국인 중도탈락 필드를 수동으로 지정
cols_add = [ cols_field2[2], '외국 ' + cols_field2[3], '외국 ' + cols_field2[4], '외국 ' + cols_field2[5], '외국 ' + cols_field2[6], '외국 ' + cols_field2[7], '외국 ' + cols_field2[8], '외국 ' + cols_field2[9], cols_field2[10] ]

print(cols_add)

In [ ]:
#df1 = df1.reindex(columns=df1.columns.to_list() + [ '외국' + cols_field2[6], '외국' + cols_field2[13], '외국' + cols_field2[14], '외국' + cols_field2[15], '외국' + cols_field2[16], '외국' + cols_field2[17], '외국' + cols_field2[18], '외국' + cols_field2[19], '외국' + cols_field2[20] ])

for n in range(len(cols_add)): #추가할 필드만큼만 루프
    print(n, len(cols_field1)+n, cols_add[n])
    df1.insert(len(cols_field1)+n, cols_add[n], 0)

In [ ]:
df1.columns

In [ ]:
df1

In [ ]:
df2

## 2. 연도+학교명 키값으로 두개의 df를 결합

- df2의 첫줄을 읽음
- df2key = 기준연도 + 학교명
- df1에서 df1key와 일치하는 레코드 확인
- 있다면.. df2의 특정 필드값을 읽어서 채움

In [ ]:
print(df1.columns)
print(df2.columns)

In [ ]:
#print(df3.loc[2])  # 첫번째 행 선택
#print(df3.loc[2, '기준연도'])  # 첫번째 행의 'Name' 열 선택
#print(df3.loc[2, '학교'])  # 첫번째 행의 'Name' 열 선택

df1key = str(df1.loc[2, '기준연도']) + df1.loc[2, '학교'] #전처리중 인덱드2줄을 삭제했기 때문에...
df2key = str(df2.loc[2, '기준연도']) + df2.loc[2, '학교']

print('df1key : ', df1key)
print('df2key : ', df2key)


In [ ]:
for m in range(len(df2)): #df2의 키와 일치하는 값을 찾음
    df2key = str(df2.loc[2+m, '기준연도']) + df2.loc[2+m, '학교']
    #print('df2key : ', df2key)

    for n in range(len(df1)): #추가할 필드만큼만 루프
        df1key = str(df1.loc[2+n, '기준연도']) + df1.loc[2+n, '학교']
        #print('df1key : ', df1key)
        
        if df1key == df2key:
            #print('df1key : ', df1key)
            #print('df2key : ', df2key)
            
            df1list = list(df1.loc[2+n]) #df1은 24개
            df2list = list(df2.loc[2+m]) #df2는 9개
            
            #print('df1list : ', df1list)
            #print('df2list : ', df2list)
            
            
            dfadd_data = df1list[0:24] + df2list[2:]
            #print('dfadd_data : ', dfadd_data)
            
            df1.loc[2+n] = dfadd_data
            
            
            break #같은 키를 찾았으니 브레이크
            
       

In [ ]:
df1

In [ ]:


# db truncate tables
# 기존 데이터를 삭제 후 인서트, 계속 추가하는 로직이면 삭제
# sql_truncate = 'truncate table {}.{}'.format(dbname, table_dropout)

# curs.execute(sql_truncate)
# conn.commit()

In [ ]:
df1

### 전처리 완료된 중도탈락 데이터를 DB에 인서트
- dbname = 'cosshubtest'  ## 변경 필요
- table_univ = 'university'    ## 변경 필요
- table_dropout = 'dropout_stat'    ## 변경 필요

In [ ]:
# db truncate tables
# 기존 데이터를 삭제 후 인서트, 계속 추가하는 로직이면 삭제
sql_truncate = 'truncate table {}.{}'.format(dbname, table_dropout)

curs.execute(sql_truncate)
conn.commit()


year, univ_cd, stud_tot, stud_sum, stud_unregist, stud_unreturn, stud_drop, stud_warn, stud_act, stud_expulsion, stud_attendyear, stud_etc, stud_rate, fresh_tot, fresh_sum, fresh_unregist, fresh_unreturn, fresh_drop, fresh_warn, fresh_act, fresh_expulsion, fresh_attendyear, fresh_etc, fresh_rate, foreign_tot, foreign_sum, foreign_degree, foreign_common, foreign_lang, foreign_exchange, foreign_visit, foreign_etc, foreign_rate

In [ ]:
# data insert to db

field_names = ''
field_data_str = ''

drop_columns = len(df1.columns) #엑셀 df읠 필드명

sql_col = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table_dropout)

curs.execute(sql_col)
#conn.commit()
col_name = curs.fetchall() #컬럼명, 리스트 안에 딕셔너리로...

col_name_list = list()
col_name_str = ''
for n in range(len(col_name)):
    col_name_list.append(col_name[n]['COLUMN_NAME'])
    
for n in range(len(col_name)):
    if n == 0:
        col_name_str = str(col_name_list[n]) 
    else:
        col_name_str = col_name_str + ', ' + str(col_name_list[n])
        

#print(col_name_str)
# print(type(col_name), len(col_name), col_name)
# print(type(col_name[0]), len(col_name[0]), col_name[0])
# print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])


for n in range(len(df1)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 인서트
    if n >= 2 : #첫 두줄은 제목행임
        field_data = list(df1.iloc[n]) # 시리즈 -> 리스트
        #print(field_data)
        #print(field_data[0])
        
        for m in range(len(col_name)):
            if m == 0: #2023
                field_data_str = str(field_data[m])            
            elif m == 1: # "0000069"
                field_data_str = field_data_str + ', "' + str(field_data[m] + '"')
            else:
                field_data_str = field_data_str + ', ' + str(field_data[m]) 
        
        print(type(field_data_str), field_data_str)
        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_dropout, col_name_str, field_data_str)
        print(n, field_data[1], sql_insert)
        curs.execute(sql_insert)
        #print(col_name_str)
        #print(field_data_str)
        
conn.commit()

# sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
# print(sql_insert)


#print(sql)
# for idx in range(len(df)):
#     	curs.execute(sql_insert, tuple(df.values[idx]))
# conn.commit()


## 이상 uiversity와 dropout_stat 테이블 생성 완료